In [1]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt 
import pymssql
import seaborn as sns
import sklearn
from sklearn.preprocessing import MinMaxScaler
import xgboost as xgb
from sklearn.model_selection import train_test_split 
from xgboost import XGBRegressor
from sklearn import metrics

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
def get_data():
    df = pd.DataFrame()
    try:
        con = pymssql.connect(user='username',password = 'password'
                 ,host='server',database='db',autocommit = True)
    except Exception as e:
        print('unable to make connection',e)
    cur=con.cursor()

    query = """
    -------------------------------------PAYMENTS AND CHARGES DATA----------------------------------
    ----------------------------charges data--------------------------------------
    IF OBJECT_ID('TEMPDB..#RCM_CHA') IS NOT NULL
    DROP TABLE #RCM_CHA
    Select pr.practice_code as Practice_Code,  MONTH(cc.DOE) as [Month], year(cc.DOE) as [Year],
    SUM(cc.Amount) as Charges_Sum
    into #RCM_CHA from Claim_Charges cc 
    join Claims c on
    cc.Claim_No = c.Claim_No and DATEDIFF(day, cc.DOE, GETDATE())<1200
    and isnull(c.Deleted,0)= 0 and isnull(cc.Deleted,0)= 0
    join Patient p on
    p.Patient_Account = c.Patient_Account
    and isnull(p.Deleted,0)= 0
    join PRACTICES pr on
    pr.Practice_Code = p.Practice_Code 
    AND ISNULL(P.DELETED,0)=0
    AND PR.IS_ACTIVE=1 AND ISNULL(PR.IS_TEST_PRACTICE,0)=0
    AND ISNULL(PR.EXCLUDE_FROM_BILLING_REPORTS,0) = 0
    AND PR.EMR_NAME NOT IN ('CHARTS PRO','PM STANDALONE','TESTINGPRACTICE')
    AND PR.PRACTICE_CODE NOT LIKE '9090%' AND PR.PRAC_NAME NOT LIKE '%TEST%'
    AND ISNULL(Pr.Deleted,0)=0
    group by pr.practice_code , MONTH(cc.DOE), YEAR(cc.DOE)
    -----------------------------------------------------------------------  
    ---------------------------------Payments data--------------------------------------
    IF OBJECT_ID('TEMPDB..#RCM_PAY') IS NOT NULL
    DROP TABLE #RCM_PAY
    Select pr.practice_code as Practice_Code,  MONTH(CP.Date_Entry) as [Month], year(CP.Date_Entry) as [Year],
    SUM(cp.Amount_Paid) as Paymnet_Sum
    into #RCM_PAY from Claim_Payments cp
    join Claims c on
    cP.Claim_No  = c.Claim_No and DATEDIFF(day, CP.Date_Entry, GETDATE())<1200
    and isnull(c.Deleted,0)= 0 and isnull(cp.Deleted,0)= 0
    join Patient p on
    p.Patient_Account = c.Patient_Account
    and isnull(p.Deleted,0)= 0
    join PRACTICES pr on
    pr.Practice_Code = p.Practice_Code 
    AND ISNULL(P.DELETED,0)=0
    AND PR.IS_ACTIVE=1 AND ISNULL(PR.IS_TEST_PRACTICE,0)=0
    AND ISNULL(PR.EXCLUDE_FROM_BILLING_REPORTS,0) = 0
    AND PR.EMR_NAME NOT IN ('CHARTS PRO','PM STANDALONE','TESTINGPRACTICE')
    AND PR.PRACTICE_CODE NOT LIKE '9090%' AND PR.PRAC_NAME NOT LIKE '%TEST%'
    AND ISNULL(Pr.Deleted,0)=0
    group by pr.practice_code , MONTH(cp.Date_Entry), YEAR(CP.Date_Entry)
    -----------------------------------------------------------------------
    IF OBJECT_ID('TEMPDB..#RCM_PCD') IS NOT NULL
    DROP TABLE #RCM_PCD
    Select C.practice_code, C.[Month],C.[Year],C.Charges_Sum,p.Paymnet_Sum
    into #RCM_PCD
    from #RCM_CHA C
    join #RCM_PAY P on
    c.Practice_Code = p.Practice_Code and c.[Month] = p.[Month] and c.[Year] = p.[Year]  
    -----------------------------------------------------------------------------------------------
    IF OBJECT_ID('TEMPDB..#RCM_DATA') IS NOT NULL
    DROP TABLE #RCM_DATA
    SELECT r.practice_code, r.[Month],r.[Year],r.Charges_Sum,r.Paymnet_Sum,
    a.AGING_0_30, a.AGING_31_60, a.AGING_61_90, a.AGING_91_120,a.AGING_120_PLUS  into #RCM_DATA
    from #RCM_PCD r
    join MONTH_END_AGING a on
    r.practice_code = a.PRACTICE_CODE and r.[Year] = a.CLOSE_YEAR and  r.[Month] = a.CLOSE_MONTH 
    order by practice_code, YEAR, MONTH 
	------------------------------------------------------------------------------------------------------------
   	IF OBJECT_ID('TEMPDB..#RCM_PREV') IS NOT NULL
    DROP TABLE #RCM_PREV
    SELECT practice_code, Month,Year,Charges_Sum,
    COALESCE(lag(Charges_Sum) over (partition by PRACTICE_CODE order by [YEAR],[MONTH]), 0) as Prev_Month_Charges,
    Paymnet_Sum, 
    COALESCE(lag(Paymnet_Sum) over (partition by PRACTICE_CODE order by [YEAR],[MONTH]), 0) as Prev_Month_Payments,
    AGING_0_30, AGING_31_60, AGING_61_90, AGING_91_120,AGING_120_PLUS  into #RCM_PREV
    from #RCM_DATA
    -----------------------------------------------------------------------------------------------------------------
   	IF OBJECT_ID('TEMPDB..#RCM_FIN') IS NOT NULL
    DROP TABLE #RCM_FIN
    SELECT practice_code, Month,Year,Charges_Sum,Prev_Month_Charges,
    COALESCE(lag(Prev_Month_Charges) over (partition by PRACTICE_CODE order by [YEAR],[MONTH]), 0) as Prev_2nd_Month_Charges,
    Paymnet_Sum, Prev_Month_Payments,
    COALESCE(lag(Prev_Month_Payments) over (partition by PRACTICE_CODE order by [YEAR],[MONTH]), 0) as Prev_2nd_Month_Payments,
    AGING_0_30, AGING_31_60, AGING_61_90, AGING_91_120,AGING_120_PLUS  into #RCM_FIN
    from #RCM_PREV
    
    Select * from #RCM_FIN order by practice_code, YEAR, MONTH 
    ---------------------------------------------------------------------------------------------------------------------------

    
     
    """ 
    
    try:
        cur.execute(query)
        df=pd.DataFrame(cur.fetchall(),columns=[x[0] for x in cur.description])
        df.columns=df.columns.str.upper()
        con.close()
    except:
        print('Error in reading data.....')
        con.close()
    return df

In [4]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [5]:
# df = get_data()

In [6]:
# df.to_csv("YEARLY_PAY_CHAR.CSV", index = False)

In [7]:
data = pd.read_csv("YEARLY_PAY_CHAR.CSV")

In [8]:
data_copy = data.copy()

In [9]:
len(data)

15216

In [10]:
data.head(50)

,PRACTICE_CODE,MONTH,YEAR,CHARGES_SUM,PREV_MONTH_CHARGES,PREV_2ND_MONTH_CHARGES,PAYMNET_SUM,PREV_MONTH_PAYMENTS,PREV_2ND_MONTH_PAYMENTS,AGING_0_30,AGING_31_60,AGING_61_90,AGING_91_120,AGING_120_PLUS
0,100,10,2019,18120.72,0.00,0.00,7485.24,0.00,0.00,47723.15,5529.94,2041.04,2271.57,3988.1601
1,100,11,2019,98578.42,18120.72,0.00,39846.26,7485.24,0.00,36599.89,6967.96,1655.20,1061.77,4712.2001
2,100,12,2019,94319.79,98578.42,18120.72,37207.94,39846.26,7485.24,35010.79,6984.46,3668.20,3117.40,5870.2801
3,100,1,2020,77354.56,94319.79,98578.42,26831.85,37207.94,39846.26,42561.52,11326.57,2097.54,2608.87,4403.1902
4,100,2,2020,105143.63,77354.56,94319.79,29557.52,26831.85,37207.94,49423.63,15254.00,6792.06,1655.89,4735.5202
5,100,3,2020,67294.84,105143.63,77354.56,33920.76,29557.52,26831.85,26190.32,11132.40,7345.34,3906.57,5615.0502
6,100,4,2020,29072.00,67294.84,105143.63,15143.49,33920.76,29557.52,16550.29,9962.74,10057.86,6453.90,7648.2502
7,100,5,2020,21518.43,29072.00,67294.84,17640.68,15143.49,33920.76,9051.62,4792.24,6801.58,6237.26,8401.3902
8,100,6,2020,51948.85,21518.43,29072.00,13622.13,17640.68,15143.49,33105.21,2953.58,2736.23,3846.09,9171.9702
9,100,7,2020,81547.41,51948.85,21518.43,27621.46,13622.13,17640.68,29368.65,3884.03,2430.37,2267.05,8678.9902


In [11]:
data['PC_RATIO'] = (data['PAYMNET_SUM']/data['CHARGES_SUM'])*100

In [12]:
data.fillna(0, inplace=True)
data.replace([np.inf, -np.inf], 0, inplace=True)

In [13]:
data = data.rename(columns = {'PRACTICE_CODE':'PRAC_CODE', 'CHARGES_SUM':'CHARG_SUM','PREV_MONTH_CHARGES':'PREV_M_CHARG','PREV_2ND_MONTH_CHARGES':'PREV_2ND_M_CHARG',
                      'PAYMNET_SUM':'PAY_SUM','PREV_MONTH_PAYMENTS':'PREV_M_PAY','PREV_2ND_MONTH_PAYMENTS':'PREV_2ND_M_PAY'})

In [14]:
# data.columns

In [15]:
data['PREV_3RD_M_CHARG'] = data.groupby('PRAC_CODE')['PREV_2ND_M_CHARG'].shift(1)
data['PREV_4TH_M_CHARG'] = data.groupby('PRAC_CODE')['PREV_3RD_M_CHARG'].shift(1)
data['PREV_5TH_M_CHARG'] = data.groupby('PRAC_CODE')['PREV_4TH_M_CHARG'].shift(1)
data['PREV_6TH_M_CHARG'] = data.groupby('PRAC_CODE')['PREV_5TH_M_CHARG'].shift(1)
data['PREV_7TH_M_CHARG'] = data.groupby('PRAC_CODE')['PREV_6TH_M_CHARG'].shift(1)
data['PREV_8TH_M_CHARG'] = data.groupby('PRAC_CODE')['PREV_7TH_M_CHARG'].shift(1)
data['PREV_9TH_M_CHARG'] = data.groupby('PRAC_CODE')['PREV_8TH_M_CHARG'].shift(1)
data['PREV_10TH_M_CHARG'] = data.groupby('PRAC_CODE')['PREV_9TH_M_CHARG'].shift(1)
data['PREV_11TH_M_CHARG'] = data.groupby('PRAC_CODE')['PREV_10TH_M_CHARG'].shift(1)
data['PREV_12TH_M_CHARG'] = data.groupby('PRAC_CODE')['PREV_11TH_M_CHARG'].shift(1)

data['PREV_13TH_M_CHARG'] = data.groupby('PRAC_CODE')['PREV_12TH_M_CHARG'].shift(1)
data['PREV_14TH_M_CHARG'] = data.groupby('PRAC_CODE')['PREV_13TH_M_CHARG'].shift(1)
data['PREV_15TH_M_CHARG'] = data.groupby('PRAC_CODE')['PREV_14TH_M_CHARG'].shift(1)
data['PREV_16TH_M_CHARG'] = data.groupby('PRAC_CODE')['PREV_15TH_M_CHARG'].shift(1)
data['PREV_17TH_M_CHARG'] = data.groupby('PRAC_CODE')['PREV_16TH_M_CHARG'].shift(1)
data['PREV_18TH_M_CHARG'] = data.groupby('PRAC_CODE')['PREV_17TH_M_CHARG'].shift(1)
data['PREV_19TH_M_CHARG'] = data.groupby('PRAC_CODE')['PREV_18TH_M_CHARG'].shift(1)
data['PREV_20TH_M_CHARG'] = data.groupby('PRAC_CODE')['PREV_19TH_M_CHARG'].shift(1)
data['PREV_21TH_M_CHARG'] = data.groupby('PRAC_CODE')['PREV_20TH_M_CHARG'].shift(1)
data['PREV_22TH_M_CHARG'] = data.groupby('PRAC_CODE')['PREV_21TH_M_CHARG'].shift(1)
data['PREV_23TH_M_CHARG'] = data.groupby('PRAC_CODE')['PREV_22TH_M_CHARG'].shift(1)
data['PREV_24TH_M_CHARG'] = data.groupby('PRAC_CODE')['PREV_23TH_M_CHARG'].shift(1)

In [16]:
data['PREV_3RD_M_PAY'] = data.groupby('PRAC_CODE')['PREV_2ND_M_PAY'].shift(1)
data['PREV_4TH_M_PAY'] = data.groupby('PRAC_CODE')['PREV_3RD_M_PAY'].shift(1)
data['PREV_5TH_M_PAY'] = data.groupby('PRAC_CODE')['PREV_4TH_M_PAY'].shift(1)
data['PREV_6TH_M_PAY'] = data.groupby('PRAC_CODE')['PREV_5TH_M_PAY'].shift(1)
data['PREV_7TH_M_PAY'] = data.groupby('PRAC_CODE')['PREV_6TH_M_PAY'].shift(1)
data['PREV_8TH_M_PAY'] = data.groupby('PRAC_CODE')['PREV_7TH_M_PAY'].shift(1)
data['PREV_9TH_M_PAY'] = data.groupby('PRAC_CODE')['PREV_8TH_M_PAY'].shift(1)
data['PREV_10TH_M_PAY'] = data.groupby('PRAC_CODE')['PREV_9TH_M_PAY'].shift(1)
data['PREV_11TH_M_PAY'] = data.groupby('PRAC_CODE')['PREV_10TH_M_PAY'].shift(1)
data['PREV_12TH_M_PAY'] = data.groupby('PRAC_CODE')['PREV_11TH_M_PAY'].shift(1)

data['PREV_13TH_M_PAY'] = data.groupby('PRAC_CODE')['PREV_12TH_M_PAY'].shift(1)
data['PREV_14TH_M_PAY'] = data.groupby('PRAC_CODE')['PREV_13TH_M_PAY'].shift(1)
data['PREV_15TH_M_PAY'] = data.groupby('PRAC_CODE')['PREV_14TH_M_PAY'].shift(1)
data['PREV_16TH_M_PAY'] = data.groupby('PRAC_CODE')['PREV_15TH_M_PAY'].shift(1)
data['PREV_17TH_M_PAY'] = data.groupby('PRAC_CODE')['PREV_16TH_M_PAY'].shift(1)
data['PREV_18TH_M_PAY'] = data.groupby('PRAC_CODE')['PREV_17TH_M_PAY'].shift(1)
data['PREV_19TH_M_PAY'] = data.groupby('PRAC_CODE')['PREV_18TH_M_PAY'].shift(1)
data['PREV_20TH_M_PAY'] = data.groupby('PRAC_CODE')['PREV_19TH_M_PAY'].shift(1)
data['PREV_21TH_M_PAY'] = data.groupby('PRAC_CODE')['PREV_20TH_M_PAY'].shift(1)
data['PREV_22TH_M_PAY'] = data.groupby('PRAC_CODE')['PREV_21TH_M_PAY'].shift(1)
data['PREV_23TH_M_PAY'] = data.groupby('PRAC_CODE')['PREV_22TH_M_PAY'].shift(1)
data['PREV_24TH_M_PAY'] = data.groupby('PRAC_CODE')['PREV_23TH_M_PAY'].shift(1)

In [17]:
data['PREV_M_AGING_0_30'] = data.groupby('PRAC_CODE')['AGING_0_30'].shift(1)
data['PREV_2M_AGING_0_30'] = data.groupby('PRAC_CODE')['PREV_M_AGING_0_30'].shift(1)
data['PREV_3M_AGING_0_30'] = data.groupby('PRAC_CODE')['PREV_2M_AGING_0_30'].shift(1)
data['PREV_4M_AGING_0_30'] = data.groupby('PRAC_CODE')['PREV_3M_AGING_0_30'].shift(1)
data['PREV_5M_AGING_0_30'] = data.groupby('PRAC_CODE')['PREV_4M_AGING_0_30'].shift(1)
data['PREV_6M_AGING_0_30'] = data.groupby('PRAC_CODE')['PREV_5M_AGING_0_30'].shift(1)
data['PREV_7M_AGING_0_30'] = data.groupby('PRAC_CODE')['PREV_6M_AGING_0_30'].shift(1)
data['PREV_8M_AGING_0_30'] = data.groupby('PRAC_CODE')['PREV_7M_AGING_0_30'].shift(1)
data['PREV_9M_AGING_0_30'] = data.groupby('PRAC_CODE')['PREV_8M_AGING_0_30'].shift(1)
data['PREV_10M_AGING_0_30'] = data.groupby('PRAC_CODE')['PREV_9M_AGING_0_30'].shift(1)
data['PREV_11M_AGING_0_30'] = data.groupby('PRAC_CODE')['PREV_10M_AGING_0_30'].shift(1)
data['PREV_12M_AGING_0_30'] = data.groupby('PRAC_CODE')['PREV_11M_AGING_0_30'].shift(1)

data['PREV_13M_AGING_0_30'] = data.groupby('PRAC_CODE')['PREV_12M_AGING_0_30'].shift(1)
data['PREV_14M_AGING_0_30'] = data.groupby('PRAC_CODE')['PREV_13M_AGING_0_30'].shift(1)
data['PREV_15M_AGING_0_30'] = data.groupby('PRAC_CODE')['PREV_14M_AGING_0_30'].shift(1)
data['PREV_16M_AGING_0_30'] = data.groupby('PRAC_CODE')['PREV_15M_AGING_0_30'].shift(1)
data['PREV_17M_AGING_0_30'] = data.groupby('PRAC_CODE')['PREV_16M_AGING_0_30'].shift(1)
data['PREV_18M_AGING_0_30'] = data.groupby('PRAC_CODE')['PREV_17M_AGING_0_30'].shift(1)
data['PREV_19M_AGING_0_30'] = data.groupby('PRAC_CODE')['PREV_18M_AGING_0_30'].shift(1)
data['PREV_20M_AGING_0_30'] = data.groupby('PRAC_CODE')['PREV_19M_AGING_0_30'].shift(1)
data['PREV_21M_AGING_0_30'] = data.groupby('PRAC_CODE')['PREV_20M_AGING_0_30'].shift(1)
data['PREV_22M_AGING_0_30'] = data.groupby('PRAC_CODE')['PREV_21M_AGING_0_30'].shift(1)
data['PREV_23M_AGING_0_30'] = data.groupby('PRAC_CODE')['PREV_22M_AGING_0_30'].shift(1)
data['PREV_24M_AGING_0_30'] = data.groupby('PRAC_CODE')['PREV_23M_AGING_0_30'].shift(1)

In [18]:
data['PREV_M_AGING_31_60'] = data.groupby('PRAC_CODE')['AGING_31_60'].shift(1)
data['PREV_2M_AGING_31_60'] = data.groupby('PRAC_CODE')['PREV_M_AGING_31_60'].shift(1)
data['PREV_3M_AGING_31_60'] = data.groupby('PRAC_CODE')['PREV_2M_AGING_31_60'].shift(1)
data['PREV_4M_AGING_31_60'] = data.groupby('PRAC_CODE')['PREV_3M_AGING_31_60'].shift(1)
data['PREV_5M_AGING_31_60'] = data.groupby('PRAC_CODE')['PREV_4M_AGING_31_60'].shift(1)
data['PREV_6M_AGING_31_60'] = data.groupby('PRAC_CODE')['PREV_5M_AGING_31_60'].shift(1)
data['PREV_7M_AGING_31_60'] = data.groupby('PRAC_CODE')['PREV_6M_AGING_31_60'].shift(1)
data['PREV_8M_AGING_31_60'] = data.groupby('PRAC_CODE')['PREV_7M_AGING_31_60'].shift(1)
data['PREV_9M_AGING_31_60'] = data.groupby('PRAC_CODE')['PREV_8M_AGING_31_60'].shift(1)
data['PREV_10M_AGING_31_60'] = data.groupby('PRAC_CODE')['PREV_9M_AGING_31_60'].shift(1)
data['PREV_11M_AGING_31_60'] = data.groupby('PRAC_CODE')['PREV_10M_AGING_31_60'].shift(1)
data['PREV_12M_AGING_31_60'] = data.groupby('PRAC_CODE')['PREV_11M_AGING_31_60'].shift(1)

data['PREV_13M_AGING_31_60'] = data.groupby('PRAC_CODE')['PREV_12M_AGING_31_60'].shift(1)
data['PREV_14M_AGING_31_60'] = data.groupby('PRAC_CODE')['PREV_13M_AGING_31_60'].shift(1)
data['PREV_15M_AGING_31_60'] = data.groupby('PRAC_CODE')['PREV_14M_AGING_31_60'].shift(1)
data['PREV_16M_AGING_31_60'] = data.groupby('PRAC_CODE')['PREV_15M_AGING_31_60'].shift(1)
data['PREV_17M_AGING_31_60'] = data.groupby('PRAC_CODE')['PREV_16M_AGING_31_60'].shift(1)
data['PREV_18M_AGING_31_60'] = data.groupby('PRAC_CODE')['PREV_17M_AGING_31_60'].shift(1)
data['PREV_19M_AGING_31_60'] = data.groupby('PRAC_CODE')['PREV_18M_AGING_31_60'].shift(1)
data['PREV_20M_AGING_31_60'] = data.groupby('PRAC_CODE')['PREV_19M_AGING_31_60'].shift(1)
data['PREV_21M_AGING_31_60'] = data.groupby('PRAC_CODE')['PREV_20M_AGING_31_60'].shift(1)
data['PREV_22M_AGING_31_60'] = data.groupby('PRAC_CODE')['PREV_21M_AGING_31_60'].shift(1)
data['PREV_23M_AGING_31_60'] = data.groupby('PRAC_CODE')['PREV_22M_AGING_31_60'].shift(1)
data['PREV_24M_AGING_31_60'] = data.groupby('PRAC_CODE')['PREV_23M_AGING_31_60'].shift(1)

In [19]:
data['PREV_M_AGING_61_90'] = data.groupby('PRAC_CODE')['AGING_61_90'].shift(1)
data['PREV_2M_AGING_61_90'] = data.groupby('PRAC_CODE')['PREV_M_AGING_61_90'].shift(1)
data['PREV_3M_AGING_61_90'] = data.groupby('PRAC_CODE')['PREV_2M_AGING_61_90'].shift(1)
data['PREV_4M_AGING_61_90'] = data.groupby('PRAC_CODE')['PREV_3M_AGING_61_90'].shift(1)
data['PREV_5M_AGING_61_90'] = data.groupby('PRAC_CODE')['PREV_4M_AGING_61_90'].shift(1)
data['PREV_6M_AGING_61_90'] = data.groupby('PRAC_CODE')['PREV_5M_AGING_61_90'].shift(1)
data['PREV_7M_AGING_61_90'] = data.groupby('PRAC_CODE')['PREV_6M_AGING_61_90'].shift(1)
data['PREV_8M_AGING_61_90'] = data.groupby('PRAC_CODE')['PREV_7M_AGING_61_90'].shift(1)
data['PREV_9M_AGING_61_90'] = data.groupby('PRAC_CODE')['PREV_8M_AGING_61_90'].shift(1)
data['PREV_10M_AGING_61_90'] = data.groupby('PRAC_CODE')['PREV_9M_AGING_61_90'].shift(1)
data['PREV_11M_AGING_61_90'] = data.groupby('PRAC_CODE')['PREV_10M_AGING_61_90'].shift(1)
data['PREV_12M_AGING_61_90'] = data.groupby('PRAC_CODE')['PREV_11M_AGING_61_90'].shift(1)

data['PREV_13M_AGING_61_90'] = data.groupby('PRAC_CODE')['PREV_12M_AGING_61_90'].shift(1)
data['PREV_14M_AGING_61_90'] = data.groupby('PRAC_CODE')['PREV_13M_AGING_61_90'].shift(1)
data['PREV_15M_AGING_61_90'] = data.groupby('PRAC_CODE')['PREV_14M_AGING_61_90'].shift(1)
data['PREV_16M_AGING_61_90'] = data.groupby('PRAC_CODE')['PREV_15M_AGING_61_90'].shift(1)
data['PREV_17M_AGING_61_90'] = data.groupby('PRAC_CODE')['PREV_16M_AGING_61_90'].shift(1)
data['PREV_18M_AGING_61_90'] = data.groupby('PRAC_CODE')['PREV_17M_AGING_61_90'].shift(1)
data['PREV_19M_AGING_61_90'] = data.groupby('PRAC_CODE')['PREV_18M_AGING_61_90'].shift(1)
data['PREV_20M_AGING_61_90'] = data.groupby('PRAC_CODE')['PREV_19M_AGING_61_90'].shift(1)
data['PREV_21M_AGING_61_90'] = data.groupby('PRAC_CODE')['PREV_20M_AGING_61_90'].shift(1)
data['PREV_22M_AGING_61_90'] = data.groupby('PRAC_CODE')['PREV_21M_AGING_61_90'].shift(1)
data['PREV_23M_AGING_61_90'] = data.groupby('PRAC_CODE')['PREV_22M_AGING_61_90'].shift(1)
data['PREV_24M_AGING_61_90'] = data.groupby('PRAC_CODE')['PREV_23M_AGING_61_90'].shift(1)

In [20]:
data['PREV_M_AGING_91_120'] = data.groupby('PRAC_CODE')['AGING_91_120'].shift(1)
data['PREV_2M_AGING_91_120'] = data.groupby('PRAC_CODE')['PREV_M_AGING_91_120'].shift(1)
data['PREV_3M_AGING_91_120'] = data.groupby('PRAC_CODE')['PREV_2M_AGING_91_120'].shift(1)
data['PREV_4M_AGING_91_120'] = data.groupby('PRAC_CODE')['PREV_3M_AGING_91_120'].shift(1)
data['PREV_5M_AGING_91_120'] = data.groupby('PRAC_CODE')['PREV_4M_AGING_91_120'].shift(1)
data['PREV_6M_AGING_91_120'] = data.groupby('PRAC_CODE')['PREV_5M_AGING_91_120'].shift(1)
data['PREV_7M_AGING_91_120'] = data.groupby('PRAC_CODE')['PREV_6M_AGING_91_120'].shift(1)
data['PREV_8M_AGING_91_120'] = data.groupby('PRAC_CODE')['PREV_7M_AGING_91_120'].shift(1)
data['PREV_9M_AGING_91_120'] = data.groupby('PRAC_CODE')['PREV_8M_AGING_91_120'].shift(1)
data['PREV_10M_AGING_91_120'] = data.groupby('PRAC_CODE')['PREV_9M_AGING_91_120'].shift(1)
data['PREV_11M_AGING_91_120'] = data.groupby('PRAC_CODE')['PREV_10M_AGING_91_120'].shift(1)
data['PREV_12M_AGING_91_120'] = data.groupby('PRAC_CODE')['PREV_11M_AGING_91_120'].shift(1)

data['PREV_13M_AGING_91_120'] = data.groupby('PRAC_CODE')['PREV_12M_AGING_91_120'].shift(1)
data['PREV_14M_AGING_91_120'] = data.groupby('PRAC_CODE')['PREV_13M_AGING_91_120'].shift(1)
data['PREV_15M_AGING_91_120'] = data.groupby('PRAC_CODE')['PREV_14M_AGING_91_120'].shift(1)
data['PREV_16M_AGING_91_120'] = data.groupby('PRAC_CODE')['PREV_15M_AGING_91_120'].shift(1)
data['PREV_17M_AGING_91_120'] = data.groupby('PRAC_CODE')['PREV_16M_AGING_91_120'].shift(1)
data['PREV_18M_AGING_91_120'] = data.groupby('PRAC_CODE')['PREV_17M_AGING_91_120'].shift(1)
data['PREV_19M_AGING_91_120'] = data.groupby('PRAC_CODE')['PREV_18M_AGING_91_120'].shift(1)
data['PREV_20M_AGING_91_120'] = data.groupby('PRAC_CODE')['PREV_19M_AGING_91_120'].shift(1)
data['PREV_21M_AGING_91_120'] = data.groupby('PRAC_CODE')['PREV_20M_AGING_91_120'].shift(1)
data['PREV_22M_AGING_91_120'] = data.groupby('PRAC_CODE')['PREV_21M_AGING_91_120'].shift(1)
data['PREV_23M_AGING_91_120'] = data.groupby('PRAC_CODE')['PREV_22M_AGING_91_120'].shift(1)
data['PREV_24M_AGING_91_120'] = data.groupby('PRAC_CODE')['PREV_23M_AGING_91_120'].shift(1)

In [21]:
data['PREV_M_AGING_120_PLUS'] = data.groupby('PRAC_CODE')['AGING_120_PLUS'].shift(1)
data['PREV_2M_AGING_120_PLUS'] = data.groupby('PRAC_CODE')['PREV_M_AGING_120_PLUS'].shift(1)
data['PREV_3M_AGING_120_PLUS'] = data.groupby('PRAC_CODE')['PREV_2M_AGING_120_PLUS'].shift(1)
data['PREV_4M_AGING_120_PLUS'] = data.groupby('PRAC_CODE')['PREV_3M_AGING_120_PLUS'].shift(1)
data['PREV_5M_AGING_120_PLUS'] = data.groupby('PRAC_CODE')['PREV_4M_AGING_120_PLUS'].shift(1)
data['PREV_6M_AGING_120_PLUS'] = data.groupby('PRAC_CODE')['PREV_5M_AGING_120_PLUS'].shift(1)
data['PREV_7M_AGING_120_PLUS'] = data.groupby('PRAC_CODE')['PREV_6M_AGING_120_PLUS'].shift(1)
data['PREV_8M_AGING_120_PLUS'] = data.groupby('PRAC_CODE')['PREV_7M_AGING_120_PLUS'].shift(1)
data['PREV_9M_AGING_120_PLUS'] = data.groupby('PRAC_CODE')['PREV_8M_AGING_120_PLUS'].shift(1)
data['PREV_10M_AGING_120_PLUS'] = data.groupby('PRAC_CODE')['PREV_9M_AGING_120_PLUS'].shift(1)
data['PREV_11M_AGING_120_PLUS'] = data.groupby('PRAC_CODE')['PREV_10M_AGING_120_PLUS'].shift(1)
data['PREV_12M_AGING_120_PLUS'] = data.groupby('PRAC_CODE')['PREV_11M_AGING_120_PLUS'].shift(1)

data['PREV_13M_AGING_120_PLUS'] = data.groupby('PRAC_CODE')['PREV_12M_AGING_120_PLUS'].shift(1)
data['PREV_14M_AGING_120_PLUS'] = data.groupby('PRAC_CODE')['PREV_13M_AGING_120_PLUS'].shift(1)
data['PREV_15M_AGING_120_PLUS'] = data.groupby('PRAC_CODE')['PREV_14M_AGING_120_PLUS'].shift(1)
data['PREV_16M_AGING_120_PLUS'] = data.groupby('PRAC_CODE')['PREV_15M_AGING_120_PLUS'].shift(1)
data['PREV_17M_AGING_120_PLUS'] = data.groupby('PRAC_CODE')['PREV_16M_AGING_120_PLUS'].shift(1)
data['PREV_18M_AGING_120_PLUS'] = data.groupby('PRAC_CODE')['PREV_17M_AGING_120_PLUS'].shift(1)
data['PREV_19M_AGING_120_PLUS'] = data.groupby('PRAC_CODE')['PREV_18M_AGING_120_PLUS'].shift(1)
data['PREV_20M_AGING_120_PLUS'] = data.groupby('PRAC_CODE')['PREV_19M_AGING_120_PLUS'].shift(1)
data['PREV_21M_AGING_120_PLUS'] = data.groupby('PRAC_CODE')['PREV_20M_AGING_120_PLUS'].shift(1)
data['PREV_22M_AGING_120_PLUS'] = data.groupby('PRAC_CODE')['PREV_21M_AGING_120_PLUS'].shift(1)
data['PREV_23M_AGING_120_PLUS'] = data.groupby('PRAC_CODE')['PREV_22M_AGING_120_PLUS'].shift(1)
data['PREV_24M_AGING_120_PLUS'] = data.groupby('PRAC_CODE')['PREV_23M_AGING_120_PLUS'].shift(1)

In [22]:
data['PREV_M_PC_RATIO'] = data.groupby('PRAC_CODE')['PC_RATIO'].shift(1)
data['PREV_2M_PC_RATIO'] = data.groupby('PRAC_CODE')['PREV_M_PC_RATIO'].shift(1)
data['PREV_3M_PC_RATIO'] = data.groupby('PRAC_CODE')['PREV_2M_PC_RATIO'].shift(1)
data['PREV_4M_PC_RATIO'] = data.groupby('PRAC_CODE')['PREV_3M_PC_RATIO'].shift(1)
data['PREV_5M_PC_RATIO'] = data.groupby('PRAC_CODE')['PREV_4M_PC_RATIO'].shift(1)
data['PREV_6M_PC_RATIO'] = data.groupby('PRAC_CODE')['PREV_5M_PC_RATIO'].shift(1)
data['PREV_7M_PC_RATIO'] = data.groupby('PRAC_CODE')['PREV_6M_PC_RATIO'].shift(1)
data['PREV_8M_PC_RATIO'] = data.groupby('PRAC_CODE')['PREV_7M_PC_RATIO'].shift(1)
data['PREV_9M_PC_RATIO'] = data.groupby('PRAC_CODE')['PREV_8M_PC_RATIO'].shift(1)
data['PREV_10M_PC_RATIO'] = data.groupby('PRAC_CODE')['PREV_9M_PC_RATIO'].shift(1)
data['PREV_11M_PC_RATIO'] = data.groupby('PRAC_CODE')['PREV_10M_PC_RATIO'].shift(1)
data['PREV_12M_PC_RATIO'] = data.groupby('PRAC_CODE')['PREV_11M_PC_RATIO'].shift(1)

data['PREV_13M_PC_RATIO'] = data.groupby('PRAC_CODE')['PREV_12M_PC_RATIO'].shift(1)
data['PREV_14M_PC_RATIO'] = data.groupby('PRAC_CODE')['PREV_13M_PC_RATIO'].shift(1)
data['PREV_15M_PC_RATIO'] = data.groupby('PRAC_CODE')['PREV_14M_PC_RATIO'].shift(1)
data['PREV_16M_PC_RATIO'] = data.groupby('PRAC_CODE')['PREV_15M_PC_RATIO'].shift(1)
data['PREV_17M_PC_RATIO'] = data.groupby('PRAC_CODE')['PREV_16M_PC_RATIO'].shift(1)
data['PREV_18M_PC_RATIO'] = data.groupby('PRAC_CODE')['PREV_17M_PC_RATIO'].shift(1)
data['PREV_19M_PC_RATIO'] = data.groupby('PRAC_CODE')['PREV_18M_PC_RATIO'].shift(1)
data['PREV_20M_PC_RATIO'] = data.groupby('PRAC_CODE')['PREV_19M_PC_RATIO'].shift(1)
data['PREV_21M_PC_RATIO'] = data.groupby('PRAC_CODE')['PREV_20M_PC_RATIO'].shift(1)
data['PREV_22M_PC_RATIO'] = data.groupby('PRAC_CODE')['PREV_21M_PC_RATIO'].shift(1)
data['PREV_23M_PC_RATIO'] = data.groupby('PRAC_CODE')['PREV_22M_PC_RATIO'].shift(1)
data['PREV_24M_PC_RATIO'] = data.groupby('PRAC_CODE')['PREV_23M_PC_RATIO'].shift(1)

In [23]:
data.tail(1)

,PRAC_CODE,MONTH,YEAR,CHARG_SUM,PREV_M_CHARG,PREV_2ND_M_CHARG,PAY_SUM,PREV_M_PAY,PREV_2ND_M_PAY,AGING_0_30,AGING_31_60,AGING_61_90,AGING_91_120,AGING_120_PLUS,PC_RATIO,PREV_3RD_M_CHARG,PREV_4TH_M_CHARG,PREV_5TH_M_CHARG,PREV_6TH_M_CHARG,PREV_7TH_M_CHARG,PREV_8TH_M_CHARG,PREV_9TH_M_CHARG,PREV_10TH_M_CHARG,PREV_11TH_M_CHARG,PREV_12TH_M_CHARG,PREV_13TH_M_CHARG,PREV_14TH_M_CHARG,PREV_15TH_M_CHARG,PREV_16TH_M_CHARG,PREV_17TH_M_CHARG,PREV_18TH_M_CHARG,PREV_19TH_M_CHARG,PREV_20TH_M_CHARG,PREV_21TH_M_CHARG,PREV_22TH_M_CHARG,PREV_23TH_M_CHARG,PREV_24TH_M_CHARG,PREV_3RD_M_PAY,PREV_4TH_M_PAY,PREV_5TH_M_PAY,PREV_6TH_M_PAY,PREV_7TH_M_PAY,PREV_8TH_M_PAY,PREV_9TH_M_PAY,PREV_10TH_M_PAY,PREV_11TH_M_PAY,PREV_12TH_M_PAY,PREV_13TH_M_PAY,PREV_14TH_M_PAY,PREV_15TH_M_PAY,PREV_16TH_M_PAY,PREV_17TH_M_PAY,PREV_18TH_M_PAY,PREV_19TH_M_PAY,PREV_20TH_M_PAY,PREV_21TH_M_PAY,PREV_22TH_M_PAY,PREV_23TH_M_PAY,PREV_24TH_M_PAY,PREV_M_AGING_0_30,PREV_2M_AGING_0_30,PREV_3M_AGING_0_30,PREV_4M_AGING_0_30,PREV_5M_AGING_0_30,PREV_6M_AGING_0_30,PREV_7M_AGING_0_30,PREV_8M_AGING_0_30,PREV_9M_AGING_0_30,PREV_10M_AGING_0_30,PREV_11M_AGING_0_30,PREV_12M_AGING_0_30,PREV_13M_AGING_0_30,PREV_14M_AGING_0_30,PREV_15M_AGING_0_30,PREV_16M_AGING_0_30,PREV_17M_AGING_0_30,PREV_18M_AGING_0_30,PREV_19M_AGING_0_30,PREV_20M_AGING_0_30,PREV_21M_AGING_0_30,PREV_22M_AGING_0_30,PREV_23M_AGING_0_30,PREV_24M_AGING_0_30,PREV_M_AGING_31_60,PREV_2M_AGING_31_60,PREV_3M_AGING_31_60,PREV_4M_AGING_31_60,PREV_5M_AGING_31_60,PREV_6M_AGING_31_60,PREV_7M_AGING_31_60,PREV_8M_AGING_31_60,PREV_9M_AGING_31_60,PREV_10M_AGING_31_60,PREV_11M_AGING_31_60,PREV_12M_AGING_31_60,PREV_13M_AGING_31_60,PREV_14M_AGING_31_60,PREV_15M_AGING_31_60,PREV_16M_AGING_31_60,PREV_17M_AGING_31_60,PREV_18M_AGING_31_60,PREV_19M_AGING_31_60,PREV_20M_AGING_31_60,PREV_21M_AGING_31_60,PREV_22M_AGING_31_60,PREV_23M_AGING_31_60,PREV_24M_AGING_31_60,PREV_M_AGING_61_90,PREV_2M_AGING_61_90,PREV_3M_AGING_61_90,PREV_4M_AGING_61_90,PREV_5M_AGING_61_90,PREV_6M_AGING_61_90,PREV_7M_AGING_61_90,PREV_8M_AGING_61_90,PREV_9M_AGING_61_90,PREV_10M_AGING_61_90,PREV_11M_AGING_61_90,PREV_12M_AGING_61_90,PREV_13M_AGING_61_90,PREV_14M_AGING_61_90,PREV_15M_AGING_61_90,PREV_16M_AGING_61_90,PREV_17M_AGING_61_90,PREV_18M_AGING_61_90,PREV_19M_AGING_61_90,PREV_20M_AGING_61_90,PREV_21M_AGING_61_90,PREV_22M_AGING_61_90,PREV_23M_AGING_61_90,PREV_24M_AGING_61_90,PREV_M_AGING_91_120,PREV_2M_AGING_91_120,PREV_3M_AGING_91_120,PREV_4M_AGING_91_120,PREV_5M_AGING_91_120,PREV_6M_AGING_91_120,PREV_7M_AGING_91_120,PREV_8M_AGING_91_120,PREV_9M_AGING_91_120,PREV_10M_AGING_91_120,PREV_11M_AGING_91_120,PREV_12M_AGING_91_120,PREV_13M_AGING_91_120,PREV_14M_AGING_91_120,PREV_15M_AGING_91_120,PREV_16M_AGING_91_120,PREV_17M_AGING_91_120,PREV_18M_AGING_91_120,PREV_19M_AGING_91_120,PREV_20M_AGING_91_120,PREV_21M_AGING_91_120,PREV_22M_AGING_91_120,PREV_23M_AGING_91_120,PREV_24M_AGING_91_120,PREV_M_AGING_120_PLUS,PREV_2M_AGING_120_PLUS,PREV_3M_AGING_120_PLUS,PREV_4M_AGING_120_PLUS,PREV_5M_AGING_120_PLUS,PREV_6M_AGING_120_PLUS,PREV_7M_AGING_120_PLUS,PREV_8M_AGING_120_PLUS,PREV_9M_AGING_120_PLUS,PREV_10M_AGING_120_PLUS,PREV_11M_AGING_120_PLUS,PREV_12M_AGING_120_PLUS,PREV_13M_AGING_120_PLUS,PREV_14M_AGING_120_PLUS,PREV_15M_AGING_120_PLUS,PREV_16M_AGING_120_PLUS,PREV_17M_AGING_120_PLUS,PREV_18M_AGING_120_PLUS,PREV_19M_AGING_120_PLUS,PREV_20M_AGING_120_PLUS,PREV_21M_AGING_120_PLUS,PREV_22M_AGING_120_PLUS,PREV_23M_AGING_120_PLUS,PREV_24M_AGING_120_PLUS,PREV_M_PC_RATIO,PREV_2M_PC_RATIO,PREV_3M_PC_RATIO,PREV_4M_PC_RATIO,PREV_5M_PC_RATIO,PREV_6M_PC_RATIO,PREV_7M_PC_RATIO,PREV_8M_PC_RATIO,PREV_9M_PC_RATIO,PREV_10M_PC_RATIO,PREV_11M_PC_RATIO,PREV_12M_PC_RATIO,PREV_13M_PC_RATIO,PREV_14M_PC_RATIO,PREV_15M_PC_RATIO,PREV_16M_PC_RATIO,PREV_17M_PC_RATIO,PREV_18M_PC_RATIO,PREV_19M_PC_RATIO,PREV_20M_PC_RATIO,PREV_21M_PC_RATIO,PREV_22M_PC_RATIO,PREV_23M_PC_RATIO,PREV_24M_PC_RATIO
15215,9091105,1,2023,1286.02,100.0,350.03,2371.0,325.0,843.0,1055.02,100.0,310.03,3433.0,164017.2,184.367273,4043.0,3959.0,155.0,550.0

## Need to exclude month which is used as input data for prediction(e.g for Jan prediction we will give Dec as Input variable and will exclude it from training data)

In [36]:
# data_train = data.loc[(data['YEAR']==2021) |(data['YEAR']==2023)|((data['YEAR']==2022) & (data['MONTH']!=12))]

In [37]:
data_train = data.loc[(data['YEAR']==2021)|((data['YEAR']==2022) & (data['MONTH']!=12))]

In [39]:
data_train.tail(1)

,PRAC_CODE,MONTH,YEAR,CHARG_SUM,PREV_M_CHARG,PREV_2ND_M_CHARG,PAY_SUM,PREV_M_PAY,PREV_2ND_M_PAY,AGING_0_30,AGING_31_60,AGING_61_90,AGING_91_120,AGING_120_PLUS,PC_RATIO,PREV_3RD_M_CHARG,PREV_4TH_M_CHARG,PREV_5TH_M_CHARG,PREV_6TH_M_CHARG,PREV_7TH_M_CHARG,PREV_8TH_M_CHARG,PREV_9TH_M_CHARG,PREV_10TH_M_CHARG,PREV_11TH_M_CHARG,PREV_12TH_M_CHARG,PREV_13TH_M_CHARG,PREV_14TH_M_CHARG,PREV_15TH_M_CHARG,PREV_16TH_M_CHARG,PREV_17TH_M_CHARG,PREV_18TH_M_CHARG,PREV_19TH_M_CHARG,PREV_20TH_M_CHARG,PREV_21TH_M_CHARG,PREV_22TH_M_CHARG,PREV_23TH_M_CHARG,PREV_24TH_M_CHARG,PREV_3RD_M_PAY,PREV_4TH_M_PAY,PREV_5TH_M_PAY,PREV_6TH_M_PAY,PREV_7TH_M_PAY,PREV_8TH_M_PAY,PREV_9TH_M_PAY,PREV_10TH_M_PAY,PREV_11TH_M_PAY,PREV_12TH_M_PAY,PREV_13TH_M_PAY,PREV_14TH_M_PAY,PREV_15TH_M_PAY,PREV_16TH_M_PAY,PREV_17TH_M_PAY,PREV_18TH_M_PAY,PREV_19TH_M_PAY,PREV_20TH_M_PAY,PREV_21TH_M_PAY,PREV_22TH_M_PAY,PREV_23TH_M_PAY,PREV_24TH_M_PAY,PREV_M_AGING_0_30,PREV_2M_AGING_0_30,PREV_3M_AGING_0_30,PREV_4M_AGING_0_30,PREV_5M_AGING_0_30,PREV_6M_AGING_0_30,PREV_7M_AGING_0_30,PREV_8M_AGING_0_30,PREV_9M_AGING_0_30,PREV_10M_AGING_0_30,PREV_11M_AGING_0_30,PREV_12M_AGING_0_30,PREV_13M_AGING_0_30,PREV_14M_AGING_0_30,PREV_15M_AGING_0_30,PREV_16M_AGING_0_30,PREV_17M_AGING_0_30,PREV_18M_AGING_0_30,PREV_19M_AGING_0_30,PREV_20M_AGING_0_30,PREV_21M_AGING_0_30,PREV_22M_AGING_0_30,PREV_23M_AGING_0_30,PREV_24M_AGING_0_30,PREV_M_AGING_31_60,PREV_2M_AGING_31_60,PREV_3M_AGING_31_60,PREV_4M_AGING_31_60,PREV_5M_AGING_31_60,PREV_6M_AGING_31_60,PREV_7M_AGING_31_60,PREV_8M_AGING_31_60,PREV_9M_AGING_31_60,PREV_10M_AGING_31_60,PREV_11M_AGING_31_60,PREV_12M_AGING_31_60,PREV_13M_AGING_31_60,PREV_14M_AGING_31_60,PREV_15M_AGING_31_60,PREV_16M_AGING_31_60,PREV_17M_AGING_31_60,PREV_18M_AGING_31_60,PREV_19M_AGING_31_60,PREV_20M_AGING_31_60,PREV_21M_AGING_31_60,PREV_22M_AGING_31_60,PREV_23M_AGING_31_60,PREV_24M_AGING_31_60,PREV_M_AGING_61_90,PREV_2M_AGING_61_90,PREV_3M_AGING_61_90,PREV_4M_AGING_61_90,PREV_5M_AGING_61_90,PREV_6M_AGING_61_90,PREV_7M_AGING_61_90,PREV_8M_AGING_61_90,PREV_9M_AGING_61_90,PREV_10M_AGING_61_90,PREV_11M_AGING_61_90,PREV_12M_AGING_61_90,PREV_13M_AGING_61_90,PREV_14M_AGING_61_90,PREV_15M_AGING_61_90,PREV_16M_AGING_61_90,PREV_17M_AGING_61_90,PREV_18M_AGING_61_90,PREV_19M_AGING_61_90,PREV_20M_AGING_61_90,PREV_21M_AGING_61_90,PREV_22M_AGING_61_90,PREV_23M_AGING_61_90,PREV_24M_AGING_61_90,PREV_M_AGING_91_120,PREV_2M_AGING_91_120,PREV_3M_AGING_91_120,PREV_4M_AGING_91_120,PREV_5M_AGING_91_120,PREV_6M_AGING_91_120,PREV_7M_AGING_91_120,PREV_8M_AGING_91_120,PREV_9M_AGING_91_120,PREV_10M_AGING_91_120,PREV_11M_AGING_91_120,PREV_12M_AGING_91_120,PREV_13M_AGING_91_120,PREV_14M_AGING_91_120,PREV_15M_AGING_91_120,PREV_16M_AGING_91_120,PREV_17M_AGING_91_120,PREV_18M_AGING_91_120,PREV_19M_AGING_91_120,PREV_20M_AGING_91_120,PREV_21M_AGING_91_120,PREV_22M_AGING_91_120,PREV_23M_AGING_91_120,PREV_24M_AGING_91_120,PREV_M_AGING_120_PLUS,PREV_2M_AGING_120_PLUS,PREV_3M_AGING_120_PLUS,PREV_4M_AGING_120_PLUS,PREV_5M_AGING_120_PLUS,PREV_6M_AGING_120_PLUS,PREV_7M_AGING_120_PLUS,PREV_8M_AGING_120_PLUS,PREV_9M_AGING_120_PLUS,PREV_10M_AGING_120_PLUS,PREV_11M_AGING_120_PLUS,PREV_12M_AGING_120_PLUS,PREV_13M_AGING_120_PLUS,PREV_14M_AGING_120_PLUS,PREV_15M_AGING_120_PLUS,PREV_16M_AGING_120_PLUS,PREV_17M_AGING_120_PLUS,PREV_18M_AGING_120_PLUS,PREV_19M_AGING_120_PLUS,PREV_20M_AGING_120_PLUS,PREV_21M_AGING_120_PLUS,PREV_22M_AGING_120_PLUS,PREV_23M_AGING_120_PLUS,PREV_24M_AGING_120_PLUS,PREV_M_PC_RATIO,PREV_2M_PC_RATIO,PREV_3M_PC_RATIO,PREV_4M_PC_RATIO,PREV_5M_PC_RATIO,PREV_6M_PC_RATIO,PREV_7M_PC_RATIO,PREV_8M_PC_RATIO,PREV_9M_PC_RATIO,PREV_10M_PC_RATIO,PREV_11M_PC_RATIO,PREV_12M_PC_RATIO,PREV_13M_PC_RATIO,PREV_14M_PC_RATIO,PREV_15M_PC_RATIO,PREV_16M_PC_RATIO,PREV_17M_PC_RATIO,PREV_18M_PC_RATIO,PREV_19M_PC_RATIO,PREV_20M_PC_RATIO,PREV_21M_PC_RATIO,PREV_22M_PC_RATIO,PREV_23M_PC_RATIO,PREV_24M_PC_RATIO
15213,9091105,11,2022,350.03,4043.0,3959.0,843.0,3490.0,92.0,285.03,3605.0,3709.0,153.0,160012.2,240.8365,155.0,550.0,4712.02,1096.0,7

In [40]:
data_train = data_train[['PRAC_CODE','MONTH','YEAR','CHARG_SUM','PREV_M_CHARG','PREV_2ND_M_CHARG','PREV_3RD_M_CHARG','PREV_4TH_M_CHARG',
'PREV_5TH_M_CHARG','PREV_6TH_M_CHARG','PREV_7TH_M_CHARG','PREV_8TH_M_CHARG', 'PREV_9TH_M_CHARG', 'PREV_10TH_M_CHARG',
'PREV_11TH_M_CHARG','PREV_12TH_M_CHARG','PREV_13TH_M_CHARG','PREV_14TH_M_CHARG','PREV_15TH_M_CHARG','PREV_16TH_M_CHARG',
'PREV_17TH_M_CHARG','PREV_18TH_M_CHARG','PREV_19TH_M_CHARG','PREV_20TH_M_CHARG','PREV_21TH_M_CHARG','PREV_22TH_M_CHARG',
'PREV_23TH_M_CHARG','PREV_24TH_M_CHARG','PAY_SUM','PREV_M_PAY', 'PREV_2ND_M_PAY','PREV_3RD_M_PAY','PREV_4TH_M_PAY',
'PREV_5TH_M_PAY','PREV_6TH_M_PAY', 'PREV_7TH_M_PAY','PREV_8TH_M_PAY', 'PREV_9TH_M_PAY', 'PREV_10TH_M_PAY',
'PREV_11TH_M_PAY','PREV_12TH_M_PAY','PREV_13TH_M_PAY','PREV_14TH_M_PAY','PREV_15TH_M_PAY','PREV_16TH_M_PAY','PREV_17TH_M_PAY',
'PREV_18TH_M_PAY','PREV_19TH_M_PAY','PREV_20TH_M_PAY','PREV_21TH_M_PAY','PREV_22TH_M_PAY','PREV_23TH_M_PAY','PREV_24TH_M_PAY',
'AGING_0_30','PREV_M_AGING_0_30','PREV_2M_AGING_0_30', 'PREV_3M_AGING_0_30','PREV_4M_AGING_0_30','PREV_5M_AGING_0_30', 'PREV_6M_AGING_0_30',
'PREV_7M_AGING_0_30','PREV_8M_AGING_0_30', 'PREV_9M_AGING_0_30','PREV_10M_AGING_0_30','PREV_11M_AGING_0_30',
'PREV_12M_AGING_0_30','PREV_13M_AGING_0_30','PREV_14M_AGING_0_30','PREV_15M_AGING_0_30','PREV_16M_AGING_0_30','PREV_17M_AGING_0_30',
'PREV_18M_AGING_0_30','PREV_19M_AGING_0_30','PREV_20M_AGING_0_30','PREV_21M_AGING_0_30','PREV_22M_AGING_0_30','PREV_23M_AGING_0_30',
'PREV_24M_AGING_0_30','AGING_31_60','PREV_M_AGING_31_60','PREV_2M_AGING_31_60', 'PREV_3M_AGING_31_60', 'PREV_4M_AGING_31_60',
'PREV_5M_AGING_31_60','PREV_6M_AGING_31_60', 'PREV_7M_AGING_31_60','PREV_8M_AGING_31_60', 'PREV_9M_AGING_31_60',
'PREV_10M_AGING_31_60','PREV_11M_AGING_31_60', 'PREV_12M_AGING_31_60','PREV_13M_AGING_31_60','PREV_14M_AGING_31_60',
'PREV_15M_AGING_31_60','PREV_16M_AGING_31_60','PREV_17M_AGING_31_60','PREV_18M_AGING_31_60','PREV_19M_AGING_31_60','PREV_20M_AGING_31_60',
'PREV_21M_AGING_31_60','PREV_22M_AGING_31_60','PREV_23M_AGING_31_60','PREV_24M_AGING_31_60','AGING_61_90','PREV_M_AGING_61_90',
'PREV_2M_AGING_61_90', 'PREV_3M_AGING_61_90', 'PREV_4M_AGING_61_90', 'PREV_5M_AGING_61_90','PREV_6M_AGING_61_90',
'PREV_7M_AGING_61_90', 'PREV_8M_AGING_61_90', 'PREV_9M_AGING_61_90', 'PREV_10M_AGING_61_90','PREV_11M_AGING_61_90',
'PREV_12M_AGING_61_90','PREV_13M_AGING_61_90','PREV_14M_AGING_61_90','PREV_15M_AGING_61_90','PREV_16M_AGING_61_90',
'PREV_17M_AGING_61_90','PREV_18M_AGING_61_90','PREV_19M_AGING_61_90','PREV_20M_AGING_61_90','PREV_21M_AGING_61_90',
'PREV_22M_AGING_61_90','PREV_23M_AGING_61_90','PREV_24M_AGING_61_90','AGING_91_120','PREV_M_AGING_91_120','PREV_2M_AGING_91_120',
'PREV_3M_AGING_91_120', 'PREV_4M_AGING_91_120','PREV_5M_AGING_91_120','PREV_6M_AGING_91_120','PREV_7M_AGING_91_120',
'PREV_8M_AGING_91_120', 'PREV_9M_AGING_91_120', 'PREV_10M_AGING_91_120','PREV_11M_AGING_91_120','PREV_12M_AGING_91_120',
'PREV_13M_AGING_91_120','PREV_14M_AGING_91_120','PREV_15M_AGING_91_120','PREV_16M_AGING_91_120','PREV_17M_AGING_91_120',
'PREV_18M_AGING_91_120','PREV_19M_AGING_91_120','PREV_20M_AGING_91_120','PREV_21M_AGING_91_120','PREV_22M_AGING_91_120',
'PREV_23M_AGING_91_120','PREV_24M_AGING_91_120','AGING_120_PLUS','PREV_M_AGING_120_PLUS','PREV_2M_AGING_120_PLUS','PREV_3M_AGING_120_PLUS', 
'PREV_4M_AGING_120_PLUS','PREV_5M_AGING_120_PLUS','PREV_6M_AGING_120_PLUS','PREV_7M_AGING_120_PLUS', 'PREV_8M_AGING_120_PLUS',
'PREV_9M_AGING_120_PLUS','PREV_10M_AGING_120_PLUS','PREV_11M_AGING_120_PLUS', 'PREV_12M_AGING_120_PLUS','PREV_13M_AGING_120_PLUS',
'PREV_14M_AGING_120_PLUS','PREV_15M_AGING_120_PLUS','PREV_16M_AGING_120_PLUS','PREV_17M_AGING_120_PLUS','PREV_18M_AGING_120_PLUS',
'PREV_19M_AGING_120_PLUS','PREV_20M_AGING_120_PLUS','PREV_21M_AGING_120_PLUS','PREV_22M_AGING_120_PLUS','PREV_23M_AGING_120_PLUS',
'PREV_24M_AGING_120_PLUS','PC_RATIO','PREV_M_PC_RATIO','PREV_2M_PC_RATIO', 'PREV_3M_PC_RATIO', 'PREV_4M_PC_RATIO','PREV_5M_PC_RATIO', 
'PREV_6M_PC_RATIO', 'PREV_7M_PC_RATIO','PREV_8M_PC_RATIO', 'PREV_9M_PC_RATIO', 'PREV_10M_PC_RATIO','PREV_11M_PC_RATIO',
'PREV_12M_PC_RATIO','PREV_13M_PC_RATIO','PREV_14M_PC_RATIO','PREV_15M_PC_RATIO','PREV_16M_PC_RATIO','PREV_17M_PC_RATIO',
'PREV_18M_PC_RATIO','PREV_19M_PC_RATIO','PREV_20M_PC_RATIO','PREV_21M_PC_RATIO','PREV_22M_PC_RATIO',
'PREV_23M_PC_RATIO','PREV_24M_PC_RATIO']]

In [41]:
data_train.fillna(0, inplace=True)
data_train.replace([np.inf, -np.inf], 0, inplace=True)

In [42]:
data_train.head(1)

,PRAC_CODE,MONTH,YEAR,CHARG_SUM,PREV_M_CHARG,PREV_2ND_M_CHARG,PREV_3RD_M_CHARG,PREV_4TH_M_CHARG,PREV_5TH_M_CHARG,PREV_6TH_M_CHARG,PREV_7TH_M_CHARG,PREV_8TH_M_CHARG,PREV_9TH_M_CHARG,PREV_10TH_M_CHARG,PREV_11TH_M_CHARG,PREV_12TH_M_CHARG,PREV_13TH_M_CHARG,PREV_14TH_M_CHARG,PREV_15TH_M_CHARG,PREV_16TH_M_CHARG,PREV_17TH_M_CHARG,PREV_18TH_M_CHARG,PREV_19TH_M_CHARG,PREV_20TH_M_CHARG,PREV_21TH_M_CHARG,PREV_22TH_M_CHARG,PREV_23TH_M_CHARG,PREV_24TH_M_CHARG,PAY_SUM,PREV_M_PAY,PREV_2ND_M_PAY,PREV_3RD_M_PAY,PREV_4TH_M_PAY,PREV_5TH_M_PAY,PREV_6TH_M_PAY,PREV_7TH_M_PAY,PREV_8TH_M_PAY,PREV_9TH_M_PAY,PREV_10TH_M_PAY,PREV_11TH_M_PAY,PREV_12TH_M_PAY,PREV_13TH_M_PAY,PREV_14TH_M_PAY,PREV_15TH_M_PAY,PREV_16TH_M_PAY,PREV_17TH_M_PAY,PREV_18TH_M_PAY,PREV_19TH_M_PAY,PREV_20TH_M_PAY,PREV_21TH_M_PAY,PREV_22TH_M_PAY,PREV_23TH_M_PAY,PREV_24TH_M_PAY,AGING_0_30,PREV_M_AGING_0_30,PREV_2M_AGING_0_30,PREV_3M_AGING_0_30,PREV_4M_AGING_0_30,PREV_5M_AGING_0_30,PREV_6M_AGING_0_30,PREV_7M_AGING_0_30,PREV_8M_AGING_0_30,PREV_9M_AGING_0_30,PREV_10M_AGING_0_30,PREV_11M_AGING_0_30,PREV_12M_AGING_0_30,PREV_13M_AGING_0_30,PREV_14M_AGING_0_30,PREV_15M_AGING_0_30,PREV_16M_AGING_0_30,PREV_17M_AGING_0_30,PREV_18M_AGING_0_30,PREV_19M_AGING_0_30,PREV_20M_AGING_0_30,PREV_21M_AGING_0_30,PREV_22M_AGING_0_30,PREV_23M_AGING_0_30,PREV_24M_AGING_0_30,AGING_31_60,PREV_M_AGING_31_60,PREV_2M_AGING_31_60,PREV_3M_AGING_31_60,PREV_4M_AGING_31_60,PREV_5M_AGING_31_60,PREV_6M_AGING_31_60,PREV_7M_AGING_31_60,PREV_8M_AGING_31_60,PREV_9M_AGING_31_60,PREV_10M_AGING_31_60,PREV_11M_AGING_31_60,PREV_12M_AGING_31_60,PREV_13M_AGING_31_60,PREV_14M_AGING_31_60,PREV_15M_AGING_31_60,PREV_16M_AGING_31_60,PREV_17M_AGING_31_60,PREV_18M_AGING_31_60,PREV_19M_AGING_31_60,PREV_20M_AGING_31_60,PREV_21M_AGING_31_60,PREV_22M_AGING_31_60,PREV_23M_AGING_31_60,PREV_24M_AGING_31_60,AGING_61_90,PREV_M_AGING_61_90,PREV_2M_AGING_61_90,PREV_3M_AGING_61_90,PREV_4M_AGING_61_90,PREV_5M_AGING_61_90,PREV_6M_AGING_61_90,PREV_7M_AGING_61_90,PREV_8M_AGING_61_90,PREV_9M_AGING_61_90,PREV_10M_AGING_61_90,PREV_11M_AGING_61_90,PREV_12M_AGING_61_90,PREV_13M_AGING_61_90,PREV_14M_AGING_61_90,PREV_15M_AGING_61_90,PREV_16M_AGING_61_90,PREV_17M_AGING_61_90,PREV_18M_AGING_61_90,PREV_19M_AGING_61_90,PREV_20M_AGING_61_90,PREV_21M_AGING_61_90,PREV_22M_AGING_61_90,PREV_23M_AGING_61_90,PREV_24M_AGING_61_90,AGING_91_120,PREV_M_AGING_91_120,PREV_2M_AGING_91_120,PREV_3M_AGING_91_120,PREV_4M_AGING_91_120,PREV_5M_AGING_91_120,PREV_6M_AGING_91_120,PREV_7M_AGING_91_120,PREV_8M_AGING_91_120,PREV_9M_AGING_91_120,PREV_10M_AGING_91_120,PREV_11M_AGING_91_120,PREV_12M_AGING_91_120,PREV_13M_AGING_91_120,PREV_14M_AGING_91_120,PREV_15M_AGING_91_120,PREV_16M_AGING_91_120,PREV_17M_AGING_91_120,PREV_18M_AGING_91_120,PREV_19M_AGING_91_120,PREV_20M_AGING_91_120,PREV_21M_AGING_91_120,PREV_22M_AGING_91_120,PREV_23M_AGING_91_120,PREV_24M_AGING_91_120,AGING_120_PLUS,PREV_M_AGING_120_PLUS,PREV_2M_AGING_120_PLUS,PREV_3M_AGING_120_PLUS,PREV_4M_AGING_120_PLUS,PREV_5M_AGING_120_PLUS,PREV_6M_AGING_120_PLUS,PREV_7M_AGING_120_PLUS,PREV_8M_AGING_120_PLUS,PREV_9M_AGING_120_PLUS,PREV_10M_AGING_120_PLUS,PREV_11M_AGING_120_PLUS,PREV_12M_AGING_120_PLUS,PREV_13M_AGING_120_PLUS,PREV_14M_AGING_120_PLUS,PREV_15M_AGING_120_PLUS,PREV_16M_AGING_120_PLUS,PREV_17M_AGING_120_PLUS,PREV_18M_AGING_120_PLUS,PREV_19M_AGING_120_PLUS,PREV_20M_AGING_120_PLUS,PREV_21M_AGING_120_PLUS,PREV_22M_AGING_120_PLUS,PREV_23M_AGING_120_PLUS,PREV_24M_AGING_120_PLUS,PC_RATIO,PREV_M_PC_RATIO,PREV_2M_PC_RATIO,PREV_3M_PC_RATIO,PREV_4M_PC_RATIO,PREV_5M_PC_RATIO,PREV_6M_PC_RATIO,PREV_7M_PC_RATIO,PREV_8M_PC_RATIO,PREV_9M_PC_RATIO,PREV_10M_PC_RATIO,PREV_11M_PC_RATIO,PREV_12M_PC_RATIO,PREV_13M_PC_RATIO,PREV_14M_PC_RATIO,PREV_15M_PC_RATIO,PREV_16M_PC_RATIO,PREV_17M_PC_RATIO,PREV_18M_PC_RATIO,PREV_19M_PC_RATIO,PREV_20M_PC_RATIO,PREV_21M_PC_RATIO,PREV_22M_PC_RATIO,PREV_23M_PC_RATIO,PREV_24M_PC_RATIO
15,100,1,2021,79764.3,61761.44,87013.06,128006.02,71746.25,62649.69,81547.41,51948.85,21518.43,29072.0,67294.84,105143.63,77354.56,943

## Payments Prediction 

In [43]:
data_train.head(1)

,PRAC_CODE,MONTH,YEAR,CHARG_SUM,PREV_M_CHARG,PREV_2ND_M_CHARG,PREV_3RD_M_CHARG,PREV_4TH_M_CHARG,PREV_5TH_M_CHARG,PREV_6TH_M_CHARG,PREV_7TH_M_CHARG,PREV_8TH_M_CHARG,PREV_9TH_M_CHARG,PREV_10TH_M_CHARG,PREV_11TH_M_CHARG,PREV_12TH_M_CHARG,PREV_13TH_M_CHARG,PREV_14TH_M_CHARG,PREV_15TH_M_CHARG,PREV_16TH_M_CHARG,PREV_17TH_M_CHARG,PREV_18TH_M_CHARG,PREV_19TH_M_CHARG,PREV_20TH_M_CHARG,PREV_21TH_M_CHARG,PREV_22TH_M_CHARG,PREV_23TH_M_CHARG,PREV_24TH_M_CHARG,PAY_SUM,PREV_M_PAY,PREV_2ND_M_PAY,PREV_3RD_M_PAY,PREV_4TH_M_PAY,PREV_5TH_M_PAY,PREV_6TH_M_PAY,PREV_7TH_M_PAY,PREV_8TH_M_PAY,PREV_9TH_M_PAY,PREV_10TH_M_PAY,PREV_11TH_M_PAY,PREV_12TH_M_PAY,PREV_13TH_M_PAY,PREV_14TH_M_PAY,PREV_15TH_M_PAY,PREV_16TH_M_PAY,PREV_17TH_M_PAY,PREV_18TH_M_PAY,PREV_19TH_M_PAY,PREV_20TH_M_PAY,PREV_21TH_M_PAY,PREV_22TH_M_PAY,PREV_23TH_M_PAY,PREV_24TH_M_PAY,AGING_0_30,PREV_M_AGING_0_30,PREV_2M_AGING_0_30,PREV_3M_AGING_0_30,PREV_4M_AGING_0_30,PREV_5M_AGING_0_30,PREV_6M_AGING_0_30,PREV_7M_AGING_0_30,PREV_8M_AGING_0_30,PREV_9M_AGING_0_30,PREV_10M_AGING_0_30,PREV_11M_AGING_0_30,PREV_12M_AGING_0_30,PREV_13M_AGING_0_30,PREV_14M_AGING_0_30,PREV_15M_AGING_0_30,PREV_16M_AGING_0_30,PREV_17M_AGING_0_30,PREV_18M_AGING_0_30,PREV_19M_AGING_0_30,PREV_20M_AGING_0_30,PREV_21M_AGING_0_30,PREV_22M_AGING_0_30,PREV_23M_AGING_0_30,PREV_24M_AGING_0_30,AGING_31_60,PREV_M_AGING_31_60,PREV_2M_AGING_31_60,PREV_3M_AGING_31_60,PREV_4M_AGING_31_60,PREV_5M_AGING_31_60,PREV_6M_AGING_31_60,PREV_7M_AGING_31_60,PREV_8M_AGING_31_60,PREV_9M_AGING_31_60,PREV_10M_AGING_31_60,PREV_11M_AGING_31_60,PREV_12M_AGING_31_60,PREV_13M_AGING_31_60,PREV_14M_AGING_31_60,PREV_15M_AGING_31_60,PREV_16M_AGING_31_60,PREV_17M_AGING_31_60,PREV_18M_AGING_31_60,PREV_19M_AGING_31_60,PREV_20M_AGING_31_60,PREV_21M_AGING_31_60,PREV_22M_AGING_31_60,PREV_23M_AGING_31_60,PREV_24M_AGING_31_60,AGING_61_90,PREV_M_AGING_61_90,PREV_2M_AGING_61_90,PREV_3M_AGING_61_90,PREV_4M_AGING_61_90,PREV_5M_AGING_61_90,PREV_6M_AGING_61_90,PREV_7M_AGING_61_90,PREV_8M_AGING_61_90,PREV_9M_AGING_61_90,PREV_10M_AGING_61_90,PREV_11M_AGING_61_90,PREV_12M_AGING_61_90,PREV_13M_AGING_61_90,PREV_14M_AGING_61_90,PREV_15M_AGING_61_90,PREV_16M_AGING_61_90,PREV_17M_AGING_61_90,PREV_18M_AGING_61_90,PREV_19M_AGING_61_90,PREV_20M_AGING_61_90,PREV_21M_AGING_61_90,PREV_22M_AGING_61_90,PREV_23M_AGING_61_90,PREV_24M_AGING_61_90,AGING_91_120,PREV_M_AGING_91_120,PREV_2M_AGING_91_120,PREV_3M_AGING_91_120,PREV_4M_AGING_91_120,PREV_5M_AGING_91_120,PREV_6M_AGING_91_120,PREV_7M_AGING_91_120,PREV_8M_AGING_91_120,PREV_9M_AGING_91_120,PREV_10M_AGING_91_120,PREV_11M_AGING_91_120,PREV_12M_AGING_91_120,PREV_13M_AGING_91_120,PREV_14M_AGING_91_120,PREV_15M_AGING_91_120,PREV_16M_AGING_91_120,PREV_17M_AGING_91_120,PREV_18M_AGING_91_120,PREV_19M_AGING_91_120,PREV_20M_AGING_91_120,PREV_21M_AGING_91_120,PREV_22M_AGING_91_120,PREV_23M_AGING_91_120,PREV_24M_AGING_91_120,AGING_120_PLUS,PREV_M_AGING_120_PLUS,PREV_2M_AGING_120_PLUS,PREV_3M_AGING_120_PLUS,PREV_4M_AGING_120_PLUS,PREV_5M_AGING_120_PLUS,PREV_6M_AGING_120_PLUS,PREV_7M_AGING_120_PLUS,PREV_8M_AGING_120_PLUS,PREV_9M_AGING_120_PLUS,PREV_10M_AGING_120_PLUS,PREV_11M_AGING_120_PLUS,PREV_12M_AGING_120_PLUS,PREV_13M_AGING_120_PLUS,PREV_14M_AGING_120_PLUS,PREV_15M_AGING_120_PLUS,PREV_16M_AGING_120_PLUS,PREV_17M_AGING_120_PLUS,PREV_18M_AGING_120_PLUS,PREV_19M_AGING_120_PLUS,PREV_20M_AGING_120_PLUS,PREV_21M_AGING_120_PLUS,PREV_22M_AGING_120_PLUS,PREV_23M_AGING_120_PLUS,PREV_24M_AGING_120_PLUS,PC_RATIO,PREV_M_PC_RATIO,PREV_2M_PC_RATIO,PREV_3M_PC_RATIO,PREV_4M_PC_RATIO,PREV_5M_PC_RATIO,PREV_6M_PC_RATIO,PREV_7M_PC_RATIO,PREV_8M_PC_RATIO,PREV_9M_PC_RATIO,PREV_10M_PC_RATIO,PREV_11M_PC_RATIO,PREV_12M_PC_RATIO,PREV_13M_PC_RATIO,PREV_14M_PC_RATIO,PREV_15M_PC_RATIO,PREV_16M_PC_RATIO,PREV_17M_PC_RATIO,PREV_18M_PC_RATIO,PREV_19M_PC_RATIO,PREV_20M_PC_RATIO,PREV_21M_PC_RATIO,PREV_22M_PC_RATIO,PREV_23M_PC_RATIO,PREV_24M_PC_RATIO
15,100,1,2021,79764.3,61761.44,87013.06,128006.02,71746.25,62649.69,81547.41,51948.85,21518.43,29072.0,67294.84,105143.63,77354.56,943

In [44]:
# Predictors=['PRAC_CODE','MONTH','YEAR','PREV_M_CHARG','PREV_2ND_M_CHARG','PREV_3RD_M_CHARG','PREV_4TH_M_CHARG',
# 'PREV_5TH_M_CHARG','PREV_6TH_M_CHARG','PREV_7TH_M_CHARG','PREV_8TH_M_CHARG', 'PREV_9TH_M_CHARG', 'PREV_10TH_M_CHARG',
# 'PREV_11TH_M_CHARG','PREV_12TH_M_CHARG','PREV_13TH_M_CHARG','PREV_14TH_M_CHARG','PREV_15TH_M_CHARG','PREV_16TH_M_CHARG',
# 'PREV_17TH_M_CHARG','PREV_18TH_M_CHARG','PREV_19TH_M_CHARG','PREV_20TH_M_CHARG','PREV_21TH_M_CHARG','PREV_22TH_M_CHARG',
# 'PREV_23TH_M_CHARG','PREV_24TH_M_CHARG','PREV_M_PAY', 'PREV_2ND_M_PAY','PREV_3RD_M_PAY','PREV_4TH_M_PAY',
# 'PREV_5TH_M_PAY','PREV_6TH_M_PAY', 'PREV_7TH_M_PAY','PREV_8TH_M_PAY', 'PREV_9TH_M_PAY', 'PREV_10TH_M_PAY',
# 'PREV_11TH_M_PAY','PREV_12TH_M_PAY','PREV_13TH_M_PAY','PREV_14TH_M_PAY','PREV_15TH_M_PAY','PREV_16TH_M_PAY','PREV_17TH_M_PAY',
# 'PREV_18TH_M_PAY','PREV_19TH_M_PAY','PREV_20TH_M_PAY','PREV_21TH_M_PAY','PREV_22TH_M_PAY','PREV_23TH_M_PAY','PREV_24TH_M_PAY',
# 'PREV_M_AGING_0_30','PREV_2M_AGING_0_30', 'PREV_3M_AGING_0_30','PREV_4M_AGING_0_30','PREV_5M_AGING_0_30', 'PREV_6M_AGING_0_30',
# 'PREV_7M_AGING_0_30','PREV_8M_AGING_0_30', 'PREV_9M_AGING_0_30','PREV_10M_AGING_0_30','PREV_11M_AGING_0_30',
# 'PREV_12M_AGING_0_30','PREV_13M_AGING_0_30','PREV_14M_AGING_0_30','PREV_15M_AGING_0_30','PREV_16M_AGING_0_30','PREV_17M_AGING_0_30',
# 'PREV_18M_AGING_0_30','PREV_19M_AGING_0_30','PREV_20M_AGING_0_30','PREV_21M_AGING_0_30','PREV_22M_AGING_0_30','PREV_23M_AGING_0_30',
# 'PREV_24M_AGING_0_30','PREV_M_AGING_31_60','PREV_2M_AGING_31_60', 'PREV_3M_AGING_31_60', 'PREV_4M_AGING_31_60',
# 'PREV_5M_AGING_31_60','PREV_6M_AGING_31_60', 'PREV_7M_AGING_31_60','PREV_8M_AGING_31_60', 'PREV_9M_AGING_31_60',
# 'PREV_10M_AGING_31_60','PREV_11M_AGING_31_60', 'PREV_12M_AGING_31_60','PREV_13M_AGING_31_60','PREV_14M_AGING_31_60',
# 'PREV_15M_AGING_31_60','PREV_16M_AGING_31_60','PREV_17M_AGING_31_60','PREV_18M_AGING_31_60','PREV_19M_AGING_31_60','PREV_20M_AGING_31_60',
# 'PREV_21M_AGING_31_60','PREV_22M_AGING_31_60','PREV_23M_AGING_31_60','PREV_24M_AGING_31_60','PREV_M_AGING_61_90',
# 'PREV_2M_AGING_61_90', 'PREV_3M_AGING_61_90', 'PREV_4M_AGING_61_90', 'PREV_5M_AGING_61_90','PREV_6M_AGING_61_90',
# 'PREV_7M_AGING_61_90', 'PREV_8M_AGING_61_90', 'PREV_9M_AGING_61_90', 'PREV_10M_AGING_61_90','PREV_11M_AGING_61_90',
# 'PREV_12M_AGING_61_90','PREV_13M_AGING_61_90','PREV_14M_AGING_61_90','PREV_15M_AGING_61_90','PREV_16M_AGING_61_90',
# 'PREV_17M_AGING_61_90','PREV_18M_AGING_61_90','PREV_19M_AGING_61_90','PREV_20M_AGING_61_90','PREV_21M_AGING_61_90',
# 'PREV_22M_AGING_61_90','PREV_23M_AGING_61_90','PREV_24M_AGING_61_90','PREV_M_AGING_91_120','PREV_2M_AGING_91_120',
# 'PREV_3M_AGING_91_120', 'PREV_4M_AGING_91_120','PREV_5M_AGING_91_120','PREV_6M_AGING_91_120','PREV_7M_AGING_91_120',
# 'PREV_8M_AGING_91_120', 'PREV_9M_AGING_91_120', 'PREV_10M_AGING_91_120','PREV_11M_AGING_91_120','PREV_12M_AGING_91_120',
# 'PREV_13M_AGING_91_120','PREV_14M_AGING_91_120','PREV_15M_AGING_91_120','PREV_16M_AGING_91_120','PREV_17M_AGING_91_120',
# 'PREV_18M_AGING_91_120','PREV_19M_AGING_91_120','PREV_20M_AGING_91_120','PREV_21M_AGING_91_120','PREV_22M_AGING_91_120',
# 'PREV_23M_AGING_91_120','PREV_24M_AGING_91_120','PREV_M_AGING_120_PLUS','PREV_2M_AGING_120_PLUS','PREV_3M_AGING_120_PLUS', 
# 'PREV_4M_AGING_120_PLUS','PREV_5M_AGING_120_PLUS','PREV_6M_AGING_120_PLUS','PREV_7M_AGING_120_PLUS', 'PREV_8M_AGING_120_PLUS',
# 'PREV_9M_AGING_120_PLUS','PREV_10M_AGING_120_PLUS','PREV_11M_AGING_120_PLUS', 'PREV_12M_AGING_120_PLUS','PREV_13M_AGING_120_PLUS',
# 'PREV_14M_AGING_120_PLUS','PREV_15M_AGING_120_PLUS','PREV_16M_AGING_120_PLUS','PREV_17M_AGING_120_PLUS','PREV_18M_AGING_120_PLUS',
# 'PREV_19M_AGING_120_PLUS','PREV_20M_AGING_120_PLUS','PREV_21M_AGING_120_PLUS','PREV_22M_AGING_120_PLUS','PREV_23M_AGING_120_PLUS',
# 'PREV_24M_AGING_120_PLUS','PREV_M_PC_RATIO','PREV_2M_PC_RATIO', 'PREV_3M_PC_RATIO', 'PREV_4M_PC_RATIO','PREV_5M_PC_RATIO', 
# 'PREV_6M_PC_RATIO', 'PREV_7M_PC_RATIO','PREV_8M_PC_RATIO', 'PREV_9M_PC_RATIO', 'PREV_10M_PC_RATIO','PREV_11M_PC_RATIO',
# 'PREV_12M_PC_RATIO','PREV_13M_PC_RATIO','PREV_14M_PC_RATIO','PREV_15M_PC_RATIO','PREV_16M_PC_RATIO','PREV_17M_PC_RATIO',
# 'PREV_18M_PC_RATIO','PREV_19M_PC_RATIO','PREV_20M_PC_RATIO','PREV_21M_PC_RATIO','PREV_22M_PC_RATIO',
# 'PREV_23M_PC_RATIO','PREV_24M_PC_RATIO']
# TargetVariable=['PAY_SUM']
# X=data_train[Predictors].values
# y=data_train[TargetVariable].values

In [45]:
Predictors=['PRAC_CODE','MONTH','YEAR','CHARG_SUM','PREV_M_CHARG','PREV_2ND_M_CHARG','PREV_3RD_M_CHARG','PREV_4TH_M_CHARG',
'PREV_5TH_M_CHARG','PREV_6TH_M_CHARG','PREV_7TH_M_CHARG','PREV_8TH_M_CHARG', 'PREV_9TH_M_CHARG', 'PREV_10TH_M_CHARG',
'PREV_11TH_M_CHARG','PREV_12TH_M_CHARG','PREV_13TH_M_CHARG','PREV_14TH_M_CHARG','PREV_15TH_M_CHARG','PREV_16TH_M_CHARG',
'PREV_17TH_M_CHARG','PREV_18TH_M_CHARG','PREV_19TH_M_CHARG','PREV_20TH_M_CHARG','PREV_21TH_M_CHARG','PREV_22TH_M_CHARG',
'PREV_23TH_M_CHARG','PREV_24TH_M_CHARG','PAY_SUM','PREV_M_PAY', 'PREV_2ND_M_PAY','PREV_3RD_M_PAY','PREV_4TH_M_PAY',
'PREV_5TH_M_PAY','PREV_6TH_M_PAY', 'PREV_7TH_M_PAY','PREV_8TH_M_PAY', 'PREV_9TH_M_PAY', 'PREV_10TH_M_PAY',
'PREV_11TH_M_PAY','PREV_12TH_M_PAY','PREV_13TH_M_PAY','PREV_14TH_M_PAY','PREV_15TH_M_PAY','PREV_16TH_M_PAY','PREV_17TH_M_PAY',
'PREV_18TH_M_PAY','PREV_19TH_M_PAY','PREV_20TH_M_PAY','PREV_21TH_M_PAY','PREV_22TH_M_PAY','PREV_23TH_M_PAY','PREV_24TH_M_PAY',
'AGING_0_30','PREV_M_AGING_0_30','PREV_2M_AGING_0_30', 'PREV_3M_AGING_0_30','PREV_4M_AGING_0_30','PREV_5M_AGING_0_30', 'PREV_6M_AGING_0_30',
'PREV_7M_AGING_0_30','PREV_8M_AGING_0_30', 'PREV_9M_AGING_0_30','PREV_10M_AGING_0_30','PREV_11M_AGING_0_30',
'PREV_12M_AGING_0_30','PREV_13M_AGING_0_30','PREV_14M_AGING_0_30','PREV_15M_AGING_0_30','PREV_16M_AGING_0_30','PREV_17M_AGING_0_30',
'PREV_18M_AGING_0_30','PREV_19M_AGING_0_30','PREV_20M_AGING_0_30','PREV_21M_AGING_0_30','PREV_22M_AGING_0_30','PREV_23M_AGING_0_30',
'PREV_24M_AGING_0_30','AGING_31_60','PREV_M_AGING_31_60','PREV_2M_AGING_31_60', 'PREV_3M_AGING_31_60', 'PREV_4M_AGING_31_60',
'PREV_5M_AGING_31_60','PREV_6M_AGING_31_60', 'PREV_7M_AGING_31_60','PREV_8M_AGING_31_60', 'PREV_9M_AGING_31_60',
'PREV_10M_AGING_31_60','PREV_11M_AGING_31_60', 'PREV_12M_AGING_31_60','PREV_13M_AGING_31_60','PREV_14M_AGING_31_60',
'PREV_15M_AGING_31_60','PREV_16M_AGING_31_60','PREV_17M_AGING_31_60','PREV_18M_AGING_31_60','PREV_19M_AGING_31_60','PREV_20M_AGING_31_60',
'PREV_21M_AGING_31_60','PREV_22M_AGING_31_60','PREV_23M_AGING_31_60','PREV_24M_AGING_31_60','AGING_61_90','PREV_M_AGING_61_90',
'PREV_2M_AGING_61_90', 'PREV_3M_AGING_61_90', 'PREV_4M_AGING_61_90', 'PREV_5M_AGING_61_90','PREV_6M_AGING_61_90',
'PREV_7M_AGING_61_90', 'PREV_8M_AGING_61_90', 'PREV_9M_AGING_61_90', 'PREV_10M_AGING_61_90','PREV_11M_AGING_61_90',
'PREV_12M_AGING_61_90','PREV_13M_AGING_61_90','PREV_14M_AGING_61_90','PREV_15M_AGING_61_90','PREV_16M_AGING_61_90',
'PREV_17M_AGING_61_90','PREV_18M_AGING_61_90','PREV_19M_AGING_61_90','PREV_20M_AGING_61_90','PREV_21M_AGING_61_90',
'PREV_22M_AGING_61_90','PREV_23M_AGING_61_90','PREV_24M_AGING_61_90','AGING_91_120','PREV_M_AGING_91_120','PREV_2M_AGING_91_120',
'PREV_3M_AGING_91_120', 'PREV_4M_AGING_91_120','PREV_5M_AGING_91_120','PREV_6M_AGING_91_120','PREV_7M_AGING_91_120',
'PREV_8M_AGING_91_120', 'PREV_9M_AGING_91_120', 'PREV_10M_AGING_91_120','PREV_11M_AGING_91_120','PREV_12M_AGING_91_120',
'PREV_13M_AGING_91_120','PREV_14M_AGING_91_120','PREV_15M_AGING_91_120','PREV_16M_AGING_91_120','PREV_17M_AGING_91_120',
'PREV_18M_AGING_91_120','PREV_19M_AGING_91_120','PREV_20M_AGING_91_120','PREV_21M_AGING_91_120','PREV_22M_AGING_91_120',
'PREV_23M_AGING_91_120','PREV_24M_AGING_91_120','AGING_120_PLUS','PREV_M_AGING_120_PLUS','PREV_2M_AGING_120_PLUS','PREV_3M_AGING_120_PLUS', 
'PREV_4M_AGING_120_PLUS','PREV_5M_AGING_120_PLUS','PREV_6M_AGING_120_PLUS','PREV_7M_AGING_120_PLUS', 'PREV_8M_AGING_120_PLUS',
'PREV_9M_AGING_120_PLUS','PREV_10M_AGING_120_PLUS','PREV_11M_AGING_120_PLUS', 'PREV_12M_AGING_120_PLUS','PREV_13M_AGING_120_PLUS',
'PREV_14M_AGING_120_PLUS','PREV_15M_AGING_120_PLUS','PREV_16M_AGING_120_PLUS','PREV_17M_AGING_120_PLUS','PREV_18M_AGING_120_PLUS',
'PREV_19M_AGING_120_PLUS','PREV_20M_AGING_120_PLUS','PREV_21M_AGING_120_PLUS','PREV_22M_AGING_120_PLUS','PREV_23M_AGING_120_PLUS',
'PREV_24M_AGING_120_PLUS','PC_RATIO','PREV_M_PC_RATIO','PREV_2M_PC_RATIO', 'PREV_3M_PC_RATIO', 'PREV_4M_PC_RATIO','PREV_5M_PC_RATIO', 
'PREV_6M_PC_RATIO', 'PREV_7M_PC_RATIO','PREV_8M_PC_RATIO', 'PREV_9M_PC_RATIO', 'PREV_10M_PC_RATIO','PREV_11M_PC_RATIO',
'PREV_12M_PC_RATIO','PREV_13M_PC_RATIO','PREV_14M_PC_RATIO','PREV_15M_PC_RATIO','PREV_16M_PC_RATIO','PREV_17M_PC_RATIO',
'PREV_18M_PC_RATIO','PREV_19M_PC_RATIO','PREV_20M_PC_RATIO','PREV_21M_PC_RATIO','PREV_22M_PC_RATIO',
'PREV_23M_PC_RATIO','PREV_24M_PC_RATIO']
TargetVariable=['PAY_SUM']
X=data_train[Predictors].values
y=data_train[TargetVariable].values

In [46]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1, random_state = 0)

In [47]:
data_train[Predictors].head()

,PRAC_CODE,MONTH,YEAR,CHARG_SUM,PREV_M_CHARG,PREV_2ND_M_CHARG,PREV_3RD_M_CHARG,PREV_4TH_M_CHARG,PREV_5TH_M_CHARG,PREV_6TH_M_CHARG,PREV_7TH_M_CHARG,PREV_8TH_M_CHARG,PREV_9TH_M_CHARG,PREV_10TH_M_CHARG,PREV_11TH_M_CHARG,PREV_12TH_M_CHARG,PREV_13TH_M_CHARG,PREV_14TH_M_CHARG,PREV_15TH_M_CHARG,PREV_16TH_M_CHARG,PREV_17TH_M_CHARG,PREV_18TH_M_CHARG,PREV_19TH_M_CHARG,PREV_20TH_M_CHARG,PREV_21TH_M_CHARG,PREV_22TH_M_CHARG,PREV_23TH_M_CHARG,PREV_24TH_M_CHARG,PAY_SUM,PREV_M_PAY,PREV_2ND_M_PAY,PREV_3RD_M_PAY,PREV_4TH_M_PAY,PREV_5TH_M_PAY,PREV_6TH_M_PAY,PREV_7TH_M_PAY,PREV_8TH_M_PAY,PREV_9TH_M_PAY,PREV_10TH_M_PAY,PREV_11TH_M_PAY,PREV_12TH_M_PAY,PREV_13TH_M_PAY,PREV_14TH_M_PAY,PREV_15TH_M_PAY,PREV_16TH_M_PAY,PREV_17TH_M_PAY,PREV_18TH_M_PAY,PREV_19TH_M_PAY,PREV_20TH_M_PAY,PREV_21TH_M_PAY,PREV_22TH_M_PAY,PREV_23TH_M_PAY,PREV_24TH_M_PAY,AGING_0_30,PREV_M_AGING_0_30,PREV_2M_AGING_0_30,PREV_3M_AGING_0_30,PREV_4M_AGING_0_30,PREV_5M_AGING_0_30,PREV_6M_AGING_0_30,PREV_7M_AGING_0_30,PREV_8M_AGING_0_30,PREV_9M_AGING_0_30,PREV_10M_AGING_0_30,PREV_11M_AGING_0_30,PREV_12M_AGING_0_30,PREV_13M_AGING_0_30,PREV_14M_AGING_0_30,PREV_15M_AGING_0_30,PREV_16M_AGING_0_30,PREV_17M_AGING_0_30,PREV_18M_AGING_0_30,PREV_19M_AGING_0_30,PREV_20M_AGING_0_30,PREV_21M_AGING_0_30,PREV_22M_AGING_0_30,PREV_23M_AGING_0_30,PREV_24M_AGING_0_30,AGING_31_60,PREV_M_AGING_31_60,PREV_2M_AGING_31_60,PREV_3M_AGING_31_60,PREV_4M_AGING_31_60,PREV_5M_AGING_31_60,PREV_6M_AGING_31_60,PREV_7M_AGING_31_60,PREV_8M_AGING_31_60,PREV_9M_AGING_31_60,PREV_10M_AGING_31_60,PREV_11M_AGING_31_60,PREV_12M_AGING_31_60,PREV_13M_AGING_31_60,PREV_14M_AGING_31_60,PREV_15M_AGING_31_60,PREV_16M_AGING_31_60,PREV_17M_AGING_31_60,PREV_18M_AGING_31_60,PREV_19M_AGING_31_60,PREV_20M_AGING_31_60,PREV_21M_AGING_31_60,PREV_22M_AGING_31_60,PREV_23M_AGING_31_60,PREV_24M_AGING_31_60,AGING_61_90,PREV_M_AGING_61_90,PREV_2M_AGING_61_90,PREV_3M_AGING_61_90,PREV_4M_AGING_61_90,PREV_5M_AGING_61_90,PREV_6M_AGING_61_90,PREV_7M_AGING_61_90,PREV_8M_AGING_61_90,PREV_9M_AGING_61_90,PREV_10M_AGING_61_90,PREV_11M_AGING_61_90,PREV_12M_AGING_61_90,PREV_13M_AGING_61_90,PREV_14M_AGING_61_90,PREV_15M_AGING_61_90,PREV_16M_AGING_61_90,PREV_17M_AGING_61_90,PREV_18M_AGING_61_90,PREV_19M_AGING_61_90,PREV_20M_AGING_61_90,PREV_21M_AGING_61_90,PREV_22M_AGING_61_90,PREV_23M_AGING_61_90,PREV_24M_AGING_61_90,AGING_91_120,PREV_M_AGING_91_120,PREV_2M_AGING_91_120,PREV_3M_AGING_91_120,PREV_4M_AGING_91_120,PREV_5M_AGING_91_120,PREV_6M_AGING_91_120,PREV_7M_AGING_91_120,PREV_8M_AGING_91_120,PREV_9M_AGING_91_120,PREV_10M_AGING_91_120,PREV_11M_AGING_91_120,PREV_12M_AGING_91_120,PREV_13M_AGING_91_120,PREV_14M_AGING_91_120,PREV_15M_AGING_91_120,PREV_16M_AGING_91_120,PREV_17M_AGING_91_120,PREV_18M_AGING_91_120,PREV_19M_AGING_91_120,PREV_20M_AGING_91_120,PREV_21M_AGING_91_120,PREV_22M_AGING_91_120,PREV_23M_AGING_91_120,PREV_24M_AGING_91_120,AGING_120_PLUS,PREV_M_AGING_120_PLUS,PREV_2M_AGING_120_PLUS,PREV_3M_AGING_120_PLUS,PREV_4M_AGING_120_PLUS,PREV_5M_AGING_120_PLUS,PREV_6M_AGING_120_PLUS,PREV_7M_AGING_120_PLUS,PREV_8M_AGING_120_PLUS,PREV_9M_AGING_120_PLUS,PREV_10M_AGING_120_PLUS,PREV_11M_AGING_120_PLUS,PREV_12M_AGING_120_PLUS,PREV_13M_AGING_120_PLUS,PREV_14M_AGING_120_PLUS,PREV_15M_AGING_120_PLUS,PREV_16M_AGING_120_PLUS,PREV_17M_AGING_120_PLUS,PREV_18M_AGING_120_PLUS,PREV_19M_AGING_120_PLUS,PREV_20M_AGING_120_PLUS,PREV_21M_AGING_120_PLUS,PREV_22M_AGING_120_PLUS,PREV_23M_AGING_120_PLUS,PREV_24M_AGING_120_PLUS,PC_RATIO,PREV_M_PC_RATIO,PREV_2M_PC_RATIO,PREV_3M_PC_RATIO,PREV_4M_PC_RATIO,PREV_5M_PC_RATIO,PREV_6M_PC_RATIO,PREV_7M_PC_RATIO,PREV_8M_PC_RATIO,PREV_9M_PC_RATIO,PREV_10M_PC_RATIO,PREV_11M_PC_RATIO,PREV_12M_PC_RATIO,PREV_13M_PC_RATIO,PREV_14M_PC_RATIO,PREV_15M_PC_RATIO,PREV_16M_PC_RATIO,PREV_17M_PC_RATIO,PREV_18M_PC_RATIO,PREV_19M_PC_RATIO,PREV_20M_PC_RATIO,PREV_21M_PC_RATIO,PREV_22M_PC_RATIO,PREV_23M_PC_RATIO,PREV_24M_PC_RATIO
15,100,1,2021,79764.30,61761.44,87013.06,128006.02,71746.25,62649.69,81547.41,51948.85,21518.43,29072.00,67294.84,105143.63,77354.56,9

In [48]:
xgb_regressor = xgb.XGBRegressor(n_estimators=1000,
    learning_rate=0.01,
    max_depth=12)

In [49]:
xgb_regressor.fit(X_train,y_train)

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, feature_types=None, gamma=0, gpu_id=-1,
             grow_policy='depthwise', importance_type=None,
             interaction_constraints='', learning_rate=0.01, max_bin=256,
             max_cat_threshold=64, max_cat_to_onehot=4, max_delta_step=0,
             max_depth=12, max_leaves=0, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=1000, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=0, ...)

## PREDICTION PART 

## Leveling Number of Rows for Input and Test set

In [50]:
data_23 = data.loc[((data['YEAR']==2023) & (data['MONTH']==1))]['PRAC_CODE']
data_23 = list(data_23)
len(data_23)

419

In [51]:
#Input data to predict future values(Need to change every month as current month)
data_pred = data.loc[(data['YEAR']==2022) & (data['MONTH']==12) & (data['PRAC_CODE'].isin(data_23))]

In [52]:
# data_pred = data.loc[(data['YEAR'] ==2022) & (data['MONTH'] ==12)]

In [53]:
len(data_pred)

406

In [54]:
data_22 = data_pred.loc[((data_pred['YEAR']==2022) & (data_pred['MONTH']==12))]['PRAC_CODE']

In [55]:
data_22 = list(data_22)
len(data_22)

406

In [56]:
data_23_test = data.loc[((data['YEAR']==2023) & (data['MONTH']==1)) & (data['PRAC_CODE'].isin(data_22))]
len(data_23_test)

406

In [57]:
# predict=['PRAC_CODE','MONTH','YEAR','PREV_M_CHARG','PREV_2ND_M_CHARG','PREV_3RD_M_CHARG','PREV_4TH_M_CHARG',
# 'PREV_5TH_M_CHARG','PREV_6TH_M_CHARG','PREV_7TH_M_CHARG','PREV_8TH_M_CHARG', 'PREV_9TH_M_CHARG', 'PREV_10TH_M_CHARG',
# 'PREV_11TH_M_CHARG','PREV_12TH_M_CHARG','PREV_13TH_M_CHARG','PREV_14TH_M_CHARG','PREV_15TH_M_CHARG','PREV_16TH_M_CHARG',
# 'PREV_17TH_M_CHARG','PREV_18TH_M_CHARG','PREV_19TH_M_CHARG','PREV_20TH_M_CHARG','PREV_21TH_M_CHARG','PREV_22TH_M_CHARG',
# 'PREV_23TH_M_CHARG','PREV_24TH_M_CHARG','PREV_M_PAY', 'PREV_2ND_M_PAY','PREV_3RD_M_PAY','PREV_4TH_M_PAY',
# 'PREV_5TH_M_PAY','PREV_6TH_M_PAY', 'PREV_7TH_M_PAY','PREV_8TH_M_PAY', 'PREV_9TH_M_PAY', 'PREV_10TH_M_PAY',
# 'PREV_11TH_M_PAY','PREV_12TH_M_PAY','PREV_13TH_M_PAY','PREV_14TH_M_PAY','PREV_15TH_M_PAY','PREV_16TH_M_PAY','PREV_17TH_M_PAY',
# 'PREV_18TH_M_PAY','PREV_19TH_M_PAY','PREV_20TH_M_PAY','PREV_21TH_M_PAY','PREV_22TH_M_PAY','PREV_23TH_M_PAY','PREV_24TH_M_PAY',
# 'PREV_M_AGING_0_30','PREV_2M_AGING_0_30', 'PREV_3M_AGING_0_30','PREV_4M_AGING_0_30','PREV_5M_AGING_0_30', 'PREV_6M_AGING_0_30',
# 'PREV_7M_AGING_0_30','PREV_8M_AGING_0_30', 'PREV_9M_AGING_0_30','PREV_10M_AGING_0_30','PREV_11M_AGING_0_30',
# 'PREV_12M_AGING_0_30','PREV_13M_AGING_0_30','PREV_14M_AGING_0_30','PREV_15M_AGING_0_30','PREV_16M_AGING_0_30','PREV_17M_AGING_0_30',
# 'PREV_18M_AGING_0_30','PREV_19M_AGING_0_30','PREV_20M_AGING_0_30','PREV_21M_AGING_0_30','PREV_22M_AGING_0_30','PREV_23M_AGING_0_30',
# 'PREV_24M_AGING_0_30','PREV_M_AGING_31_60','PREV_2M_AGING_31_60', 'PREV_3M_AGING_31_60', 'PREV_4M_AGING_31_60',
# 'PREV_5M_AGING_31_60','PREV_6M_AGING_31_60', 'PREV_7M_AGING_31_60','PREV_8M_AGING_31_60', 'PREV_9M_AGING_31_60',
# 'PREV_10M_AGING_31_60','PREV_11M_AGING_31_60', 'PREV_12M_AGING_31_60','PREV_13M_AGING_31_60','PREV_14M_AGING_31_60',
# 'PREV_15M_AGING_31_60','PREV_16M_AGING_31_60','PREV_17M_AGING_31_60','PREV_18M_AGING_31_60','PREV_19M_AGING_31_60','PREV_20M_AGING_31_60',
# 'PREV_21M_AGING_31_60','PREV_22M_AGING_31_60','PREV_23M_AGING_31_60','PREV_24M_AGING_31_60','PREV_M_AGING_61_90',
# 'PREV_2M_AGING_61_90', 'PREV_3M_AGING_61_90', 'PREV_4M_AGING_61_90', 'PREV_5M_AGING_61_90','PREV_6M_AGING_61_90',
# 'PREV_7M_AGING_61_90', 'PREV_8M_AGING_61_90', 'PREV_9M_AGING_61_90', 'PREV_10M_AGING_61_90','PREV_11M_AGING_61_90',
# 'PREV_12M_AGING_61_90','PREV_13M_AGING_61_90','PREV_14M_AGING_61_90','PREV_15M_AGING_61_90','PREV_16M_AGING_61_90',
# 'PREV_17M_AGING_61_90','PREV_18M_AGING_61_90','PREV_19M_AGING_61_90','PREV_20M_AGING_61_90','PREV_21M_AGING_61_90',
# 'PREV_22M_AGING_61_90','PREV_23M_AGING_61_90','PREV_24M_AGING_61_90','PREV_M_AGING_91_120','PREV_2M_AGING_91_120',
# 'PREV_3M_AGING_91_120', 'PREV_4M_AGING_91_120','PREV_5M_AGING_91_120','PREV_6M_AGING_91_120','PREV_7M_AGING_91_120',
# 'PREV_8M_AGING_91_120', 'PREV_9M_AGING_91_120', 'PREV_10M_AGING_91_120','PREV_11M_AGING_91_120','PREV_12M_AGING_91_120',
# 'PREV_13M_AGING_91_120','PREV_14M_AGING_91_120','PREV_15M_AGING_91_120','PREV_16M_AGING_91_120','PREV_17M_AGING_91_120',
# 'PREV_18M_AGING_91_120','PREV_19M_AGING_91_120','PREV_20M_AGING_91_120','PREV_21M_AGING_91_120','PREV_22M_AGING_91_120',
# 'PREV_23M_AGING_91_120','PREV_24M_AGING_91_120','PREV_M_AGING_120_PLUS','PREV_2M_AGING_120_PLUS','PREV_3M_AGING_120_PLUS', 
# 'PREV_4M_AGING_120_PLUS','PREV_5M_AGING_120_PLUS','PREV_6M_AGING_120_PLUS','PREV_7M_AGING_120_PLUS', 'PREV_8M_AGING_120_PLUS',
# 'PREV_9M_AGING_120_PLUS','PREV_10M_AGING_120_PLUS','PREV_11M_AGING_120_PLUS', 'PREV_12M_AGING_120_PLUS','PREV_13M_AGING_120_PLUS',
# 'PREV_14M_AGING_120_PLUS','PREV_15M_AGING_120_PLUS','PREV_16M_AGING_120_PLUS','PREV_17M_AGING_120_PLUS','PREV_18M_AGING_120_PLUS',
# 'PREV_19M_AGING_120_PLUS','PREV_20M_AGING_120_PLUS','PREV_21M_AGING_120_PLUS','PREV_22M_AGING_120_PLUS','PREV_23M_AGING_120_PLUS',
# 'PREV_24M_AGING_120_PLUS','PREV_M_PC_RATIO','PREV_2M_PC_RATIO', 'PREV_3M_PC_RATIO', 'PREV_4M_PC_RATIO','PREV_5M_PC_RATIO', 
# 'PREV_6M_PC_RATIO', 'PREV_7M_PC_RATIO','PREV_8M_PC_RATIO', 'PREV_9M_PC_RATIO', 'PREV_10M_PC_RATIO','PREV_11M_PC_RATIO',
# 'PREV_12M_PC_RATIO','PREV_13M_PC_RATIO','PREV_14M_PC_RATIO','PREV_15M_PC_RATIO','PREV_16M_PC_RATIO','PREV_17M_PC_RATIO',
# 'PREV_18M_PC_RATIO','PREV_19M_PC_RATIO','PREV_20M_PC_RATIO','PREV_21M_PC_RATIO','PREV_22M_PC_RATIO',
# 'PREV_23M_PC_RATIO','PREV_24M_PC_RATIO']


In [58]:
# data_pred = data_pred[['PRAC_CODE','MONTH','YEAR','PREV_M_CHARG','PREV_2ND_M_CHARG','PREV_3RD_M_CHARG','PREV_4TH_M_CHARG',
# 'PREV_5TH_M_CHARG','PREV_6TH_M_CHARG','PREV_7TH_M_CHARG','PREV_8TH_M_CHARG', 'PREV_9TH_M_CHARG', 'PREV_10TH_M_CHARG',
# 'PREV_11TH_M_CHARG','PREV_12TH_M_CHARG','PREV_13TH_M_CHARG','PREV_14TH_M_CHARG','PREV_15TH_M_CHARG','PREV_16TH_M_CHARG',
# 'PREV_17TH_M_CHARG','PREV_18TH_M_CHARG','PREV_19TH_M_CHARG','PREV_20TH_M_CHARG','PREV_21TH_M_CHARG','PREV_22TH_M_CHARG',
# 'PREV_23TH_M_CHARG','PREV_24TH_M_CHARG','PREV_M_PAY', 'PREV_2ND_M_PAY','PREV_3RD_M_PAY','PREV_4TH_M_PAY',
# 'PREV_5TH_M_PAY','PREV_6TH_M_PAY', 'PREV_7TH_M_PAY','PREV_8TH_M_PAY', 'PREV_9TH_M_PAY', 'PREV_10TH_M_PAY',
# 'PREV_11TH_M_PAY','PREV_12TH_M_PAY','PREV_13TH_M_PAY','PREV_14TH_M_PAY','PREV_15TH_M_PAY','PREV_16TH_M_PAY','PREV_17TH_M_PAY',
# 'PREV_18TH_M_PAY','PREV_19TH_M_PAY','PREV_20TH_M_PAY','PREV_21TH_M_PAY','PREV_22TH_M_PAY','PREV_23TH_M_PAY','PREV_24TH_M_PAY',
# 'PREV_M_AGING_0_30','PREV_2M_AGING_0_30', 'PREV_3M_AGING_0_30','PREV_4M_AGING_0_30','PREV_5M_AGING_0_30', 'PREV_6M_AGING_0_30',
# 'PREV_7M_AGING_0_30','PREV_8M_AGING_0_30', 'PREV_9M_AGING_0_30','PREV_10M_AGING_0_30','PREV_11M_AGING_0_30',
# 'PREV_12M_AGING_0_30','PREV_13M_AGING_0_30','PREV_14M_AGING_0_30','PREV_15M_AGING_0_30','PREV_16M_AGING_0_30','PREV_17M_AGING_0_30',
# 'PREV_18M_AGING_0_30','PREV_19M_AGING_0_30','PREV_20M_AGING_0_30','PREV_21M_AGING_0_30','PREV_22M_AGING_0_30','PREV_23M_AGING_0_30',
# 'PREV_24M_AGING_0_30','PREV_M_AGING_31_60','PREV_2M_AGING_31_60', 'PREV_3M_AGING_31_60', 'PREV_4M_AGING_31_60',
# 'PREV_5M_AGING_31_60','PREV_6M_AGING_31_60', 'PREV_7M_AGING_31_60','PREV_8M_AGING_31_60', 'PREV_9M_AGING_31_60',
# 'PREV_10M_AGING_31_60','PREV_11M_AGING_31_60', 'PREV_12M_AGING_31_60','PREV_13M_AGING_31_60','PREV_14M_AGING_31_60',
# 'PREV_15M_AGING_31_60','PREV_16M_AGING_31_60','PREV_17M_AGING_31_60','PREV_18M_AGING_31_60','PREV_19M_AGING_31_60','PREV_20M_AGING_31_60',
# 'PREV_21M_AGING_31_60','PREV_22M_AGING_31_60','PREV_23M_AGING_31_60','PREV_24M_AGING_31_60','PREV_M_AGING_61_90',
# 'PREV_2M_AGING_61_90', 'PREV_3M_AGING_61_90', 'PREV_4M_AGING_61_90', 'PREV_5M_AGING_61_90','PREV_6M_AGING_61_90',
# 'PREV_7M_AGING_61_90', 'PREV_8M_AGING_61_90', 'PREV_9M_AGING_61_90', 'PREV_10M_AGING_61_90','PREV_11M_AGING_61_90',
# 'PREV_12M_AGING_61_90','PREV_13M_AGING_61_90','PREV_14M_AGING_61_90','PREV_15M_AGING_61_90','PREV_16M_AGING_61_90',
# 'PREV_17M_AGING_61_90','PREV_18M_AGING_61_90','PREV_19M_AGING_61_90','PREV_20M_AGING_61_90','PREV_21M_AGING_61_90',
# 'PREV_22M_AGING_61_90','PREV_23M_AGING_61_90','PREV_24M_AGING_61_90','PREV_M_AGING_91_120','PREV_2M_AGING_91_120',
# 'PREV_3M_AGING_91_120', 'PREV_4M_AGING_91_120','PREV_5M_AGING_91_120','PREV_6M_AGING_91_120','PREV_7M_AGING_91_120',
# 'PREV_8M_AGING_91_120', 'PREV_9M_AGING_91_120', 'PREV_10M_AGING_91_120','PREV_11M_AGING_91_120','PREV_12M_AGING_91_120',
# 'PREV_13M_AGING_91_120','PREV_14M_AGING_91_120','PREV_15M_AGING_91_120','PREV_16M_AGING_91_120','PREV_17M_AGING_91_120',
# 'PREV_18M_AGING_91_120','PREV_19M_AGING_91_120','PREV_20M_AGING_91_120','PREV_21M_AGING_91_120','PREV_22M_AGING_91_120',
# 'PREV_23M_AGING_91_120','PREV_24M_AGING_91_120','PREV_M_AGING_120_PLUS','PREV_2M_AGING_120_PLUS','PREV_3M_AGING_120_PLUS', 
# 'PREV_4M_AGING_120_PLUS','PREV_5M_AGING_120_PLUS','PREV_6M_AGING_120_PLUS','PREV_7M_AGING_120_PLUS', 'PREV_8M_AGING_120_PLUS',
# 'PREV_9M_AGING_120_PLUS','PREV_10M_AGING_120_PLUS','PREV_11M_AGING_120_PLUS', 'PREV_12M_AGING_120_PLUS','PREV_13M_AGING_120_PLUS',
# 'PREV_14M_AGING_120_PLUS','PREV_15M_AGING_120_PLUS','PREV_16M_AGING_120_PLUS','PREV_17M_AGING_120_PLUS','PREV_18M_AGING_120_PLUS',
# 'PREV_19M_AGING_120_PLUS','PREV_20M_AGING_120_PLUS','PREV_21M_AGING_120_PLUS','PREV_22M_AGING_120_PLUS','PREV_23M_AGING_120_PLUS',
# 'PREV_24M_AGING_120_PLUS','PREV_M_PC_RATIO','PREV_2M_PC_RATIO', 'PREV_3M_PC_RATIO', 'PREV_4M_PC_RATIO','PREV_5M_PC_RATIO', 
# 'PREV_6M_PC_RATIO', 'PREV_7M_PC_RATIO','PREV_8M_PC_RATIO', 'PREV_9M_PC_RATIO', 'PREV_10M_PC_RATIO','PREV_11M_PC_RATIO',
# 'PREV_12M_PC_RATIO','PREV_13M_PC_RATIO','PREV_14M_PC_RATIO','PREV_15M_PC_RATIO','PREV_16M_PC_RATIO','PREV_17M_PC_RATIO',
# 'PREV_18M_PC_RATIO','PREV_19M_PC_RATIO','PREV_20M_PC_RATIO','PREV_21M_PC_RATIO','PREV_22M_PC_RATIO',
# 'PREV_23M_PC_RATIO','PREV_24M_PC_RATIO'
# ]]

In [59]:
data_pred = data_pred[['PRAC_CODE','MONTH','YEAR','CHARG_SUM','PREV_M_CHARG','PREV_2ND_M_CHARG','PREV_3RD_M_CHARG','PREV_4TH_M_CHARG',
'PREV_5TH_M_CHARG','PREV_6TH_M_CHARG','PREV_7TH_M_CHARG','PREV_8TH_M_CHARG', 'PREV_9TH_M_CHARG', 'PREV_10TH_M_CHARG',
'PREV_11TH_M_CHARG','PREV_12TH_M_CHARG','PREV_13TH_M_CHARG','PREV_14TH_M_CHARG','PREV_15TH_M_CHARG','PREV_16TH_M_CHARG',
'PREV_17TH_M_CHARG','PREV_18TH_M_CHARG','PREV_19TH_M_CHARG','PREV_20TH_M_CHARG','PREV_21TH_M_CHARG','PREV_22TH_M_CHARG',
'PREV_23TH_M_CHARG','PREV_24TH_M_CHARG','PAY_SUM','PREV_M_PAY', 'PREV_2ND_M_PAY','PREV_3RD_M_PAY','PREV_4TH_M_PAY',
'PREV_5TH_M_PAY','PREV_6TH_M_PAY', 'PREV_7TH_M_PAY','PREV_8TH_M_PAY', 'PREV_9TH_M_PAY', 'PREV_10TH_M_PAY',
'PREV_11TH_M_PAY','PREV_12TH_M_PAY','PREV_13TH_M_PAY','PREV_14TH_M_PAY','PREV_15TH_M_PAY','PREV_16TH_M_PAY','PREV_17TH_M_PAY',
'PREV_18TH_M_PAY','PREV_19TH_M_PAY','PREV_20TH_M_PAY','PREV_21TH_M_PAY','PREV_22TH_M_PAY','PREV_23TH_M_PAY','PREV_24TH_M_PAY',
'AGING_0_30','PREV_M_AGING_0_30','PREV_2M_AGING_0_30', 'PREV_3M_AGING_0_30','PREV_4M_AGING_0_30','PREV_5M_AGING_0_30', 'PREV_6M_AGING_0_30',
'PREV_7M_AGING_0_30','PREV_8M_AGING_0_30', 'PREV_9M_AGING_0_30','PREV_10M_AGING_0_30','PREV_11M_AGING_0_30',
'PREV_12M_AGING_0_30','PREV_13M_AGING_0_30','PREV_14M_AGING_0_30','PREV_15M_AGING_0_30','PREV_16M_AGING_0_30','PREV_17M_AGING_0_30',
'PREV_18M_AGING_0_30','PREV_19M_AGING_0_30','PREV_20M_AGING_0_30','PREV_21M_AGING_0_30','PREV_22M_AGING_0_30','PREV_23M_AGING_0_30',
'PREV_24M_AGING_0_30','AGING_31_60','PREV_M_AGING_31_60','PREV_2M_AGING_31_60', 'PREV_3M_AGING_31_60', 'PREV_4M_AGING_31_60',
'PREV_5M_AGING_31_60','PREV_6M_AGING_31_60', 'PREV_7M_AGING_31_60','PREV_8M_AGING_31_60', 'PREV_9M_AGING_31_60',
'PREV_10M_AGING_31_60','PREV_11M_AGING_31_60', 'PREV_12M_AGING_31_60','PREV_13M_AGING_31_60','PREV_14M_AGING_31_60',
'PREV_15M_AGING_31_60','PREV_16M_AGING_31_60','PREV_17M_AGING_31_60','PREV_18M_AGING_31_60','PREV_19M_AGING_31_60','PREV_20M_AGING_31_60',
'PREV_21M_AGING_31_60','PREV_22M_AGING_31_60','PREV_23M_AGING_31_60','PREV_24M_AGING_31_60','AGING_61_90','PREV_M_AGING_61_90',
'PREV_2M_AGING_61_90', 'PREV_3M_AGING_61_90', 'PREV_4M_AGING_61_90', 'PREV_5M_AGING_61_90','PREV_6M_AGING_61_90',
'PREV_7M_AGING_61_90', 'PREV_8M_AGING_61_90', 'PREV_9M_AGING_61_90', 'PREV_10M_AGING_61_90','PREV_11M_AGING_61_90',
'PREV_12M_AGING_61_90','PREV_13M_AGING_61_90','PREV_14M_AGING_61_90','PREV_15M_AGING_61_90','PREV_16M_AGING_61_90',
'PREV_17M_AGING_61_90','PREV_18M_AGING_61_90','PREV_19M_AGING_61_90','PREV_20M_AGING_61_90','PREV_21M_AGING_61_90',
'PREV_22M_AGING_61_90','PREV_23M_AGING_61_90','PREV_24M_AGING_61_90','AGING_91_120','PREV_M_AGING_91_120','PREV_2M_AGING_91_120',
'PREV_3M_AGING_91_120', 'PREV_4M_AGING_91_120','PREV_5M_AGING_91_120','PREV_6M_AGING_91_120','PREV_7M_AGING_91_120',
'PREV_8M_AGING_91_120', 'PREV_9M_AGING_91_120', 'PREV_10M_AGING_91_120','PREV_11M_AGING_91_120','PREV_12M_AGING_91_120',
'PREV_13M_AGING_91_120','PREV_14M_AGING_91_120','PREV_15M_AGING_91_120','PREV_16M_AGING_91_120','PREV_17M_AGING_91_120',
'PREV_18M_AGING_91_120','PREV_19M_AGING_91_120','PREV_20M_AGING_91_120','PREV_21M_AGING_91_120','PREV_22M_AGING_91_120',
'PREV_23M_AGING_91_120','PREV_24M_AGING_91_120','AGING_120_PLUS','PREV_M_AGING_120_PLUS','PREV_2M_AGING_120_PLUS','PREV_3M_AGING_120_PLUS', 
'PREV_4M_AGING_120_PLUS','PREV_5M_AGING_120_PLUS','PREV_6M_AGING_120_PLUS','PREV_7M_AGING_120_PLUS', 'PREV_8M_AGING_120_PLUS',
'PREV_9M_AGING_120_PLUS','PREV_10M_AGING_120_PLUS','PREV_11M_AGING_120_PLUS', 'PREV_12M_AGING_120_PLUS','PREV_13M_AGING_120_PLUS',
'PREV_14M_AGING_120_PLUS','PREV_15M_AGING_120_PLUS','PREV_16M_AGING_120_PLUS','PREV_17M_AGING_120_PLUS','PREV_18M_AGING_120_PLUS',
'PREV_19M_AGING_120_PLUS','PREV_20M_AGING_120_PLUS','PREV_21M_AGING_120_PLUS','PREV_22M_AGING_120_PLUS','PREV_23M_AGING_120_PLUS',
'PREV_24M_AGING_120_PLUS','PC_RATIO','PREV_M_PC_RATIO','PREV_2M_PC_RATIO', 'PREV_3M_PC_RATIO', 'PREV_4M_PC_RATIO','PREV_5M_PC_RATIO', 
'PREV_6M_PC_RATIO', 'PREV_7M_PC_RATIO','PREV_8M_PC_RATIO', 'PREV_9M_PC_RATIO', 'PREV_10M_PC_RATIO','PREV_11M_PC_RATIO',
'PREV_12M_PC_RATIO','PREV_13M_PC_RATIO','PREV_14M_PC_RATIO','PREV_15M_PC_RATIO','PREV_16M_PC_RATIO','PREV_17M_PC_RATIO',
'PREV_18M_PC_RATIO','PREV_19M_PC_RATIO','PREV_20M_PC_RATIO','PREV_21M_PC_RATIO','PREV_22M_PC_RATIO',
'PREV_23M_PC_RATIO','PREV_24M_PC_RATIO'
]]

In [60]:
pre = xgb_regressor.predict(data_pred)

In [61]:
len(pre)

406

In [62]:
prediction_pay = pd.DataFrame(pre,columns=['PREDICTED_PAYMENTS'])

In [63]:
actual = data_23_test

In [64]:
actual_pay = actual[['PRAC_CODE','MONTH','YEAR','PAY_SUM']]

In [65]:
actual_pay['PREDICTED_PAYMENTS'] = prediction_pay['PREDICTED_PAYMENTS'].to_list()

In [66]:
actual_pay['PREDICTED_PAYMENTS']= actual_pay['PREDICTED_PAYMENTS'].apply(lambda x:int(x))
actual_pay['PAY_SUM']= actual_pay['PAY_SUM'].apply(lambda x:int(x))

In [67]:
our_pred = actual_pay.copy()

In [68]:
actual_pay['VAR_PAY_%']=(((actual_pay['PAY_SUM']-actual_pay['PREDICTED_PAYMENTS'])/actual_pay['PREDICTED_PAYMENTS'])*100).round(1)

In [69]:
actual_pay

,PRAC_CODE,MONTH,YEAR,PAY_SUM,PREDICTED_PAYMENTS,VAR_PAY_%
36,100,1,2023,57483,72337,-20.5
73,111,1,2023,28475,64418,-55.8
110,133,1,2023,6442,10400,-38.1
147,138,1,2023,33266,40447,-17.8
184,182,1,2023,59333,82246,-27.9
221,189,1,2023,91586,110909,-17.4
258,198,1,2023,21032,23206,-9.4
295,204,1,2023,27539,45703,-39.7
332,205,1,2023,22404,35575,-37.0
369,206,1,2023,17530,26673,-34.3


In [70]:
act = actual_pay.PAY_SUM.sum()
pred = actual_pay.PREDICTED_PAYMENTS.sum()

In [71]:
accur = (act/pred)

In [72]:
accur

0.6665112241143177

In [73]:
finance = pd.read_excel("finance_prediction_jan.xlsx")

In [74]:
finance = finance.drop([0])
# finance.drop(columns = 'Unnamed: 0')

In [75]:
finance.head(1)

,Prac_Code,Charges,Payments
1,111,57773.18899,29431.470692


In [76]:
finance = finance.rename(columns = {'Prac_Code':'PRAC_CODE','Charges':'FINANCE_CHARGES','Payments':'FINANCE_PAYMENTS' })

In [77]:
finance = finance.fillna(0)

In [78]:
finance.head()

,PRAC_CODE,FINANCE_CHARGES,FINANCE_PAYMENTS
1,111,57773.188990,29431.470692
2,133,25387.158521,11481.291071
3,138,85831.617146,41031.929408
4,182,120315.765911,63199.078619
5,189,0.000000,0.000000


In [79]:
our_pred.head()

,PRAC_CODE,MONTH,YEAR,PAY_SUM,PREDICTED_PAYMENTS
36,100,1,2023,57483,72337
73,111,1,2023,28475,64418
110,133,1,2023,6442,10400
147,138,1,2023,33266,40447
184,182,1,2023,59333,82246


In [80]:
all_pred_pay = our_pred.merge(finance, left_on='PRAC_CODE', right_on='PRAC_CODE', how = 'inner')

In [81]:
all_pred_pay.FINANCE_PAYMENTS = all_pred_pay.FINANCE_PAYMENTS.fillna(0)

In [82]:
all_pred_pay.head(2)

,PRAC_CODE,MONTH,YEAR,PAY_SUM,PREDICTED_PAYMENTS,FINANCE_CHARGES,FINANCE_PAYMENTS
0,111,1,2023,28475,64418,57773.188990,29431.470692
1,133,1,2023,6442,10400,25387.158521,11481.291071


In [83]:
all_pred_pay.FINANCE_PAYMENTS = all_pred_pay.FINANCE_PAYMENTS.apply(int)

In [84]:
all_pred_pay = all_pred_pay.drop(columns = 'FINANCE_CHARGES')

In [85]:
all_pred_pay['VAR_PAY_%']=(((all_pred_pay['PAY_SUM']-all_pred_pay['PREDICTED_PAYMENTS'])/all_pred_pay['PREDICTED_PAYMENTS'])*100).round(1)

In [86]:
all_pred_pay['FIN_VAR_PAY_%']=(((all_pred_pay['PAY_SUM']-all_pred_pay['FINANCE_PAYMENTS'])/all_pred_pay['FINANCE_PAYMENTS'])*100).round(1)

In [87]:
all_pred_pay = all_pred_pay.loc[(all_pred_pay['PREDICTED_PAYMENTS']!=0)&(all_pred_pay['FINANCE_PAYMENTS']!=0)]

In [88]:
all_pred_pay.sort_values(by = 'PRAC_CODE')

,PRAC_CODE,MONTH,YEAR,PAY_SUM,PREDICTED_PAYMENTS,FINANCE_PAYMENTS,VAR_PAY_%,FIN_VAR_PAY_%
0,111,1,2023,28475,64418,29431,-55.8,-3.2
1,133,1,2023,6442,10400,11481,-38.1,-43.9
2,138,1,2023,33266,40447,41031,-17.8,-18.9
3,182,1,2023,59333,82246,63199,-27.9,-6.1
5,198,1,2023,21032,23206,23230,-9.4,-9.5
6,204,1,2023,27539,45703,23753,-39.7,15.9
7,205,1,2023,22404,35575,27524,-37.0,-18.6
8,206,1,2023,17530,26673,24003,-34.3,-27.0
9,207,1,2023,27308,54281,39317,-49.7,-30.5
10,219,1,2023,50624,67242,54857,-24.7,-7.7


In [89]:
# all_pred_pay.to_csv("predicted_payments.csv")

In [90]:
# all_pred_pay.loc[all_pred_pay['PRAC_CODE'] == 1012714]

In [91]:
actuall = all_pred_pay['PAY_SUM'].sum()-12456258
pred = all_pred_pay['PREDICTED_PAYMENTS'].sum()-24586490

In [92]:
(actuall/pred)*100

86.12161617944115

## Overall Practices Percentage 

## Data Science Part 

In [93]:
char80= all_pred_pay.loc[all_pred_pay['VAR_PAY_%'].between(-20,20)]
char80['PREDICTED_PAYMENTS']= char80['PREDICTED_PAYMENTS'].apply(int)
char80 = char80.sort_values(by = 'VAR_PAY_%', ascending=True).reset_index(drop=True)

In [94]:
char_70_80_Per= all_pred_pay.loc[(all_pred_pay['VAR_PAY_%'].between(-30,-20.01))|(all_pred_pay['VAR_PAY_%'].between(20.01,30))]
char_70_80_Per['PREDICTED_PAYMENTS']= char_70_80_Per['PREDICTED_PAYMENTS'].apply(int)
char_70_80_Per = char_70_80_Per.sort_values(by = 'VAR_PAY_%', ascending=False).reset_index(drop=True)

In [95]:
char_50_70= all_pred_pay.loc[(all_pred_pay['VAR_PAY_%'].between(-50,-29.999))|(all_pred_pay['VAR_PAY_%'].between(30.01,50))]
char_50_70['PREDICTED_PAYMENTS']= char_50_70['PREDICTED_PAYMENTS'].apply(int)
char_50_70 = char_50_70.sort_values(by = 'VAR_PAY_%', ascending=False).reset_index(drop=True)

In [96]:
char_b_50= all_pred_pay.loc[(all_pred_pay['VAR_PAY_%']<-50) | (all_pred_pay['VAR_PAY_%']>50)]
char_b_50['PREDICTED_PAYMENTS']= char_b_50['PREDICTED_PAYMENTS'].apply(int)
char_b_50 = char_b_50.sort_values(by = 'VAR_PAY_%', ascending=False).reset_index(drop=True)

In [97]:
print("Total practices =",all_pred_pay['PRAC_CODE'].count())
print("Practices with 80% Plus Accuracy =",char80['PRAC_CODE'].count())
print("Practices with 70 to 80% Accuracy =",char_70_80_Per['PRAC_CODE'].count())
print("Practices with 50 to 70% Accuracy =",char_50_70['PRAC_CODE'].count())
print("Practices with below 50% Accuracy =",char_b_50['PRAC_CODE'].count())

Total practices = 309
Practices with 80% Plus Accuracy = 113
Practices with 70 to 80% Accuracy = 66
Practices with 50 to 70% Accuracy = 77
Practices with below 50% Accuracy = 54


In [ ]:
125+53+81+50

In [ ]:
107+58+87+57

## Finance Part 

In [ ]:
fin_char80= all_pred_pay.loc[all_pred_pay['FIN_VAR_PAY_%'].between(-20,20)]
fin_char80['FINANCE_PAYMENTS']= fin_char80['FINANCE_PAYMENTS'].apply(int)
fin_char80 = fin_char80.sort_values(by = 'VAR_PAY_%', ascending=True).reset_index(drop=True)

In [ ]:
fin_char_70_80_Per= all_pred_pay.loc[(all_pred_pay['FIN_VAR_PAY_%'].between(-30,-20.01))|(all_pred_pay['FIN_VAR_PAY_%'].between(20.01,30))]
fin_char_70_80_Per['FINANCE_PAYMENTS']= fin_char_70_80_Per['FINANCE_PAYMENTS'].apply(int)
fin_char_70_80_Per = fin_char_70_80_Per.sort_values(by = 'FIN_VAR_PAY_%', ascending=False).reset_index(drop=True)

In [ ]:
fin_char_50_70 = all_pred_pay.loc[(all_pred_pay['FIN_VAR_PAY_%'].between(-50,-29.999))|(all_pred_pay['FIN_VAR_PAY_%'].between(30.01,50))]
fin_char_50_70['FINANCE_PAYMENTS'] = fin_char_50_70['FINANCE_PAYMENTS'].apply(int)
fin_char_50_70 = fin_char_50_70.sort_values(by = 'FIN_VAR_PAY_%', ascending=False).reset_index(drop=True)

In [ ]:
fin_char_b_50 = all_pred_pay.loc[(all_pred_pay['FIN_VAR_PAY_%']<-50) | (all_pred_pay['FIN_VAR_PAY_%']>50)]
fin_char_b_50['FINANCE_PAYMENTS'] = fin_char_b_50['FINANCE_PAYMENTS'].apply(int)
fin_char_b_50 = fin_char_b_50.sort_values(by = 'FIN_VAR_PAY_%', ascending=False).reset_index(drop=True)

In [ ]:
print("Total practices =",all_pred_pay['PRAC_CODE'].count())
print("Practices with 80% Plus Accuracy =",fin_char80['PRAC_CODE'].count())
print("Practices with 70 to 80% Accuracy =",fin_char_70_80_Per['PRAC_CODE'].count())
print("Practices with 50 to 70% Accuracy =",fin_char_50_70['PRAC_CODE'].count())
print("Practices with below 50% Accuracy =",fin_char_b_50['PRAC_CODE'].count())

In [ ]:
114+55+70+70